In [1]:
import pandas as pd
import numpy as np
import re
import pandas as pd

In [19]:
import pandas as pd

file_path_train = r"C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/Train.csv"
df = pd.read_csv(file_path_train)
# Convert 'saledate' to datetime and extract year, month, and day
df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')
df['sale_year'] = df['saledate'].dt.year
df['sale_month'] = df['saledate'].dt.month
df['sale_day'] = df['saledate'].dt.day

# Filter the dataset to only include the last 5 years of data
recent_years = df['sale_year'].dropna().unique()
recent_years.sort()
selected_years = recent_years[-5:]
df_filtered = df[df['sale_year'].isin(selected_years)]

# Drop irrelevant columns: 'SalesID', 'saledate', and high-cardinality object columns
high_cardinality_cols = [col for col in df_filtered.select_dtypes(include=['object']).columns if df_filtered[col].nunique() > 50]
df_filtered = df_filtered.drop(columns=['SalesID', 'saledate'] + high_cardinality_cols, errors='ignore')

# Drop rows with missing SalePrice (target variable)
df_filtered = df_filtered.dropna(subset=['SalePrice'])

# Identify categorical columns and convert them to category type
categorical_cols = df_filtered.select_dtypes(include=['object']).columns
df_filtered[categorical_cols] = df_filtered[categorical_cols].astype('category')

# Fill missing values: numerical columns with median, categorical with mode
for col in df_filtered.select_dtypes(include=['number']).columns:
    df_filtered[col] = df_filtered[col].fillna(df_filtered[col].median())

for col in categorical_cols:
    df_filtered[col] = df_filtered[col].fillna(df_filtered[col].mode()[0])

# Encode categorical variables
df_encoded = pd.get_dummies(df_filtered, drop_first=True)

# Split into features and target
X = df_encoded.drop(columns=['SalePrice'])
y = df_encoded['SalePrice']

# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

# Display model performance
mae, rmse


C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\3239662035.py:4: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path_train)


KeyboardInterrupt: 

In [68]:
def RMSE(y_, y_pred_):
    '''
    RSME
    '''
    return ((y_ - y_pred_) ** 2).mean() ** 0.5

In [67]:
# Optional: Log feature importances
importances = rf_model.feature_importances_

# Create a zip of feature names and feature importances
feature_zip = zip(rf_model.feature_names_in_, rf_model.feature_importances_)

# Sort the zip by feature importance in descending order
sorted_feature_zip = sorted(feature_zip, key=lambda x: x[1], reverse=True)

# Print the sorted feature names, importances, and index
for idx, (feat, importance) in enumerate(sorted_feature_zip, 1):
  print({f"feature_{idx}_{feat}_importance" : importance})


{'feature_1_Enclosure_EROPS w AC_importance': 0.2892118595873668}
{'feature_2_YearMade_importance': 0.12487402704702251}
{'feature_3_ModelID_importance': 0.11829126119004313}
{'feature_4_MachineID_importance': 0.04573237305124742}
{'feature_5_Ripper_None or Unspecified_importance': 0.03802813823731736}
{'feature_6_sale_year_importance': 0.03607100932771089}
{'feature_7_ProductGroup_SSL_importance': 0.03221605291181244}
{'feature_8_ProductSize_Mini_importance': 0.031867151945836086}
{'feature_9_ProductGroupDesc_Skid Steer Loaders_importance': 0.029094141942740615}
{'feature_10_ProductSize_Large / Medium_importance': 0.023168544653347077}
{'feature_11_sale_month_importance': 0.01967121459907647}
{'feature_12_sale_day_importance': 0.018800011754424743}
{'feature_13_Tire_Size_None or Unspecified_importance': 0.015933707555163393}
{'feature_14_Stick_Standard_importance': 0.015125249963802044}
{'feature_15_MachineHoursCurrentMeter_importance': 0.011092164781110333}
{'feature_16_ProductGroup_

In [17]:
# Re-import necessary libraries due to execution reset
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder


ProductSize_path=r'C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/ProductSize_Null_fix_model.pkl'
file_path_train = r"C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/Train.csv"
df = pd.read_csv(file_path_train)

# Convert 'saledate' to datetime and extract year, month, and day
df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')
df['sale_year'] = df['saledate'].dt.year
df['sale_month'] = df['saledate'].dt.month
df['sale_day'] = df['saledate'].dt.day

# Filter dataset to only include the last 5 years
recent_years = df['sale_year'].dropna().unique()
recent_years.sort()
selected_years = recent_years[-5:]
df_filtered = df[df['sale_year'].isin(selected_years)]

# Drop irrelevant columns: 'SalesID', 'saledate'
# Drop irrelevant columns: 'SalesID', 'saledate', and high-cardinality object columns
high_cardinality_cols = [col for col in df_filtered.select_dtypes(include=['object']).columns if df_filtered[col].nunique() > 50]
df_filtered = df_filtered.drop(columns=['SalesID', 'saledate'] + high_cardinality_cols, errors='ignore')

# Identify categorical columns
categorical_cols = df_filtered.select_dtypes(include=['object']).columns

# Identify high cardinality categorical columns (≥ 50 unique values)
high_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() >= 50]
moderate_cardinality_cols = [col for col in categorical_cols if 10 <= df_filtered[col].nunique() < 50]
low_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() < 10]

# Fill missing categorical values with "Unknown"
for col in categorical_cols:
    df_filtered[col] = df_filtered[col].fillna("Unknown")

# Encoding strategy

# Apply One-Hot Encoding (OHE) to low-cardinality categories
df_encoded = pd.get_dummies(df_filtered, columns=low_cardinality_cols, drop_first=True)

# Apply Ordinal Encoding to moderate-cardinality categories
if moderate_cardinality_cols:
    ord_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    df_encoded[moderate_cardinality_cols] = ord_encoder.fit_transform(df_encoded[moderate_cardinality_cols])

# Apply Frequency Encoding to high-cardinality categories
for col in high_cardinality_cols:
    freq_encoding = df_encoded[col].value_counts().to_dict()
    df_encoded[col] = df_encoded[col].map(freq_encoding)

# Drop rows with missing SalePrice (target variable)
df_encoded = df_encoded.dropna(subset=['SalePrice'])

# Split into features and target
X = df_encoded.drop(columns=['SalePrice'])
y = df_encoded['SalePrice']

# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

# Display model performance
mae, rmse

C:\Users\eitanb\AppData\Local\Temp\ipykernel_31536\255710431.py:8: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path_train)


(5036.5769087707195, 8135.890307052758)

In [63]:
# Optional: Log feature importances
importances = rf_model.feature_importances_

# Create a zip of feature names and feature importances
feature_zip = zip(rf_model.feature_names_in_, rf_model.feature_importances_)

# Sort the zip by feature importance in descending order
sorted_feature_zip = sorted(feature_zip, key=lambda x: x[1], reverse=True)

# Print the sorted feature names, importances, and index
for idx, (feat, importance) in enumerate(sorted_feature_zip, 1):
  print({f"feature_{idx}_{feat}_importance" : importance})


{'feature_1_Enclosure_EROPS w AC_importance': 0.2892118595873668}
{'feature_2_YearMade_importance': 0.12487402704702251}
{'feature_3_ModelID_importance': 0.11829126119004313}
{'feature_4_MachineID_importance': 0.04573237305124742}
{'feature_5_Ripper_None or Unspecified_importance': 0.03802813823731736}
{'feature_6_sale_year_importance': 0.03607100932771089}
{'feature_7_ProductGroup_SSL_importance': 0.03221605291181244}
{'feature_8_ProductSize_Mini_importance': 0.031867151945836086}
{'feature_9_ProductGroupDesc_Skid Steer Loaders_importance': 0.029094141942740615}
{'feature_10_ProductSize_Large / Medium_importance': 0.023168544653347077}
{'feature_11_sale_month_importance': 0.01967121459907647}
{'feature_12_sale_day_importance': 0.018800011754424743}
{'feature_13_Tire_Size_None or Unspecified_importance': 0.015933707555163393}
{'feature_14_Stick_Standard_importance': 0.015125249963802044}
{'feature_15_MachineHoursCurrentMeter_importance': 0.011092164781110333}
{'feature_16_ProductGroup_

In [21]:
def calc_YearMade(df, model_id):
    valid_years = df.loc[(df['ModelID'] == model_id) & (df['YearMade'] > 1000), 'YearMade']
    return int(valid_years.median()) if not valid_years.empty else None  # Return None if no valid years exist

def update_YearMade(df):
    print("update_YearMade START")
    
    # Compute median YearMade for each ModelID
    model_medians = df.loc[df['YearMade'] > 1000].groupby('ModelID')['YearMade'].median()
    
    # Update YearMade where it is 1000 using ModelID median
    df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
    
    # Compute overall median YearMade excluding 1000 values
    overall_median = df.loc[df['YearMade'] > 1000, 'YearMade'].median()
    
    # Replace any remaining 1000 values with overall median
    df['YearMade'].fillna(overall_median, inplace=True)
        
    # Bucketize YearMade
    bins = [0, 1980, 1995, 2005, 2010]
    labels = ["Before 1980", "1980-1995", "1996-2005", "2006-2010"]
    df['YearMade_Bucket'] = pd.cut(df['YearMade'], bins=bins, labels=labels, right=True)
    
    print("update_YearMade END")
    return df

In [26]:
# Re-import necessary libraries due to execution reset
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder


# Reload the dataset
ProductSize_path=r'C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/ProductSize_Null_fix_model.pkl'
file_path_train = r"C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/Train.csv"
df = pd.read_csv(file_path_train)

# Convert 'saledate' to datetime and extract year, month, and day
df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')
df['sale_year'] = df['saledate'].dt.year
df['sale_month'] = df['saledate'].dt.month
df['sale_day'] = df['saledate'].dt.day

# Define function to update 'YearMade' and create 'YearMade_Bucket'
def update_YearMade(df):
    print("update_YearMade START")
    
    # Compute median YearMade for each ModelID
    model_medians = df.loc[df['YearMade'] > 1000].groupby('ModelID')['YearMade'].median()
    
    # Update YearMade where it is 1000 using ModelID median
    df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
    
    # Compute overall median YearMade excluding 1000 values
    overall_median = df.loc[df['YearMade'] > 1000, 'YearMade'].median()
    
    # Replace any remaining 1000 values with overall median
    df['YearMade'].fillna(overall_median, inplace=True)
        
    # Bucketize YearMade
    bins = [0, 1980, 1995, 2005, 2010, 2025]
    labels = ["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]
    df['YearMade_Bucket'] = pd.cut(df['YearMade'], bins=bins, labels=labels, right=True)
    
    print("update_YearMade END")
    return df

# Apply YearMade update
df = update_YearMade(df)

# Filter dataset to only include the last 5 years
recent_years = df['sale_year'].dropna().unique()
recent_years.sort()
selected_years = recent_years[-5:]
df_filtered = df[df['sale_year'].isin(selected_years)]

# Drop irrelevant columns: 'SalesID', 'saledate'
high_cardinality_cols = [col for col in df_filtered.select_dtypes(include=['object']).columns if df_filtered[col].nunique() > 50]
df_filtered = df_filtered.drop(columns=['SalesID', 'saledate'] + high_cardinality_cols, errors='ignore')

# Identify categorical columns
categorical_cols = df_filtered.select_dtypes(include=['object']).columns

# Apply Ordinal Encoding to 'YearMade_Bucket'
year_bucket_encoder = OrdinalEncoder(categories=[["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]])
df_filtered["YearMade_Bucket"] = year_bucket_encoder.fit_transform(df_filtered[["YearMade_Bucket"]])

# Identify high, moderate, and low cardinality categorical columns
high_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() >= 50]
moderate_cardinality_cols = [col for col in categorical_cols if 10 <= df_filtered[col].nunique() < 50]
low_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() < 10]

# Fill missing categorical values with "Unknown"
for col in categorical_cols:
    df_filtered[col] = df_filtered[col].fillna("Unknown")

# Encoding strategy

# Apply One-Hot Encoding (OHE) to low-cardinality categories
df_encoded = pd.get_dummies(df_filtered, columns=low_cardinality_cols, drop_first=True)

# Apply Ordinal Encoding to moderate-cardinality categories
if moderate_cardinality_cols:
    ord_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    df_encoded[moderate_cardinality_cols] = ord_encoder.fit_transform(df_encoded[moderate_cardinality_cols])

# Apply Frequency Encoding to high-cardinality categories
for col in high_cardinality_cols:
    freq_encoding = df_encoded[col].value_counts().to_dict()
    df_encoded[col] = df_encoded[col].map(freq_encoding)

# Drop rows with missing SalePrice (target variable)
df_encoded = df_encoded.dropna(subset=['SalePrice'])

# Split into features and target
X = df_encoded.drop(columns=['SalePrice'])
y = df_encoded['SalePrice']

# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

# Display model performance
mae, rmse

C:\Users\eitanb\AppData\Local\Temp\ipykernel_31536\4082236078.py:9: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path_train)


update_YearMade START


C:\Users\eitanb\AppData\Local\Temp\ipykernel_31536\4082236078.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1980. 1980. 1994. ... 2003. 2004. 2004.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
C:\Users\eitanb\AppData\Local\Temp\ipykernel_31536\4082236078.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['YearM

update_YearMade END


(4901.082367667127, 7917.90265227888)

In [28]:
def preprocess_product_size(df):
    """
    Function to preprocess and impute missing ProductSize values based on fiProductClassDesc.
    
    Steps:
    1. Extract Product Type and Metric Tons / Horsepower from fiProductClassDesc.
    2. Identify Unit Type (Metric Tons, Horsepower, or Lb Operating Capacity).
    3. Convert Metric Tons / Horsepower to numerical values.
    4. Use Metric Tons to impute missing ProductSize values.
    """
    
    # Step 1: Extract Product Type and Size Descriptor
    def split_product_desc(desc):
        if pd.isna(desc):
            return pd.NA, pd.NA
        match = re.search(r'(.+?)-\s*([\d\.]+.*)', desc)
        if match:
            return match.group(1).strip(), match.group(2).strip()
        return desc.strip(), pd.NA  # If no match, return full description as product
    
    df[['Product_Type', 'Metric_Tons_HP']] = df['fiProductClassDesc'].apply(lambda x: pd.Series(split_product_desc(x)))
    
    # Step 2: Identify Unit Type
    def extract_unit_type(desc):
        if pd.isna(desc):
            return pd.NA
        if "Metric Tons" in desc:
            return "Metric Tons"
        elif "Horsepower" in desc:
            return "Horsepower"
        elif "Lb Operating Capacity" in desc:
            return "Lb Operating Capacity"
        return pd.NA
    
    df["Unit_Type"] = df["Metric_Tons_HP"].apply(extract_unit_type)
    
    # Step 3: Convert Metric Tons to numerical values
    def extract_numeric_range(value):
        """Extracts the average value from a range like '12.0 to 14.0 Metric Tons'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:  # If range exists, take the average
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:  # If only one number exists, use it
            return float(numbers[0])
        return np.nan
    
    df['Metric_Tons_Value'] = df['Metric_Tons_HP'].apply(lambda x: extract_numeric_range(x) if isinstance(x, str) else np.nan)
    
    # Step 4: Impute missing ProductSize values using Metric Tons
    def impute_product_size(row):
        if pd.isna(row['ProductSize']) and not pd.isna(row['Metric_Tons_Value']):
            if row['Metric_Tons_Value'] <= 5:
                return 'Mini'
            elif 5 < row['Metric_Tons_Value'] <= 75:
                return 'Compact'
            elif 20 < row['Metric_Tons_Value'] <= 50:
                return 'Large / Medium'
            elif 75 < row['Metric_Tons_Value'] <= 200:
                return 'Medium'
            elif row['Metric_Tons_Value'] > 200:
                return 'Large'
        return row['ProductSize']
    
    df['ProductSize_Imputed'] = df.apply(impute_product_size, axis=1)
    df.drop(columns='ProductSize',inplace=True)
    
    return df

In [29]:
# Re-import necessary libraries due to execution reset
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder


# Reload the dataset
ProductSize_path=r'C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/ProductSize_Null_fix_model.pkl'
file_path_train = r"C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/Train.csv"
df = pd.read_csv(file_path_train)

# Convert 'saledate' to datetime and extract year, month, and day
df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')
df['sale_year'] = df['saledate'].dt.year
df['sale_month'] = df['saledate'].dt.month
df['sale_day'] = df['saledate'].dt.day
def preprocess_product_size(df):
    """
    Function to preprocess and impute missing ProductSize values based on fiProductClassDesc.
    
    Steps:
    1. Extract Product Type and Metric Tons / Horsepower from fiProductClassDesc.
    2. Identify Unit Type (Metric Tons, Horsepower, or Lb Operating Capacity).
    3. Convert Metric Tons / Horsepower to numerical values.
    4. Use Metric Tons to impute missing ProductSize values.
    """
    
    # Step 1: Extract Product Type and Size Descriptor
    def split_product_desc(desc):
        if pd.isna(desc):
            return pd.NA, pd.NA
        match = re.search(r'(.+?)-\s*([\d\.]+.*)', desc)
        if match:
            return match.group(1).strip(), match.group(2).strip()
        return desc.strip(), pd.NA  # If no match, return full description as product
    
    df[['Product_Type', 'Metric_Tons_HP']] = df['fiProductClassDesc'].apply(lambda x: pd.Series(split_product_desc(x)))
    
    # Step 2: Identify Unit Type
    def extract_unit_type(desc):
        if pd.isna(desc):
            return pd.NA
        if "Metric Tons" in desc:
            return "Metric Tons"
        elif "Horsepower" in desc:
            return "Horsepower"
        elif "Lb Operating Capacity" in desc:
            return "Lb Operating Capacity"
        return pd.NA
    
    df["Unit_Type"] = df["Metric_Tons_HP"].apply(extract_unit_type)
    
    # Step 3: Convert Metric Tons to numerical values
    def extract_numeric_range(value):
        """Extracts the average value from a range like '12.0 to 14.0 Metric Tons'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:  # If range exists, take the average
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:  # If only one number exists, use it
            return float(numbers[0])
        return np.nan
    
    df['Metric_Tons_Value'] = df['Metric_Tons_HP'].apply(lambda x: extract_numeric_range(x) if isinstance(x, str) else np.nan)
    
    # Step 4: Impute missing ProductSize values using Metric Tons
    def impute_product_size(row):
        if pd.isna(row['ProductSize']) and not pd.isna(row['Metric_Tons_Value']):
            if row['Metric_Tons_Value'] <= 5:
                return 'Mini'
            elif 5 < row['Metric_Tons_Value'] <= 75:
                return 'Compact'
            elif 20 < row['Metric_Tons_Value'] <= 50:
                return 'Large / Medium'
            elif 75 < row['Metric_Tons_Value'] <= 200:
                return 'Medium'
            elif row['Metric_Tons_Value'] > 200:
                return 'Large'
        return row['ProductSize']
    
    df['ProductSize_Imputed'] = df.apply(impute_product_size, axis=1)
    df.drop(columns='ProductSize',inplace=True)
    
    return df
df=preprocess_product_size(df)
# Define function to update 'YearMade' and create 'YearMade_Bucket'
def update_YearMade(df):
    print("update_YearMade START")
    
    # Compute median YearMade for each ModelID
    model_medians = df.loc[df['YearMade'] > 1000].groupby('ModelID')['YearMade'].median()
    
    # Update YearMade where it is 1000 using ModelID median
    df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
    
    # Compute overall median YearMade excluding 1000 values
    overall_median = df.loc[df['YearMade'] > 1000, 'YearMade'].median()
    
    # Replace any remaining 1000 values with overall median
    df['YearMade'].fillna(overall_median, inplace=True)
        
    # Bucketize YearMade
    bins = [0, 1980, 1995, 2005, 2010, 2025]
    labels = ["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]
    df['YearMade_Bucket'] = pd.cut(df['YearMade'], bins=bins, labels=labels, right=True)
    
    print("update_YearMade END")
    return df

# Apply YearMade update
df = update_YearMade(df)

# Filter dataset to only include the last 5 years
recent_years = df['sale_year'].dropna().unique()
recent_years.sort()
selected_years = recent_years[-5:]
df_filtered = df[df['sale_year'].isin(selected_years)]

# Drop irrelevant columns: 'SalesID', 'saledate'
high_cardinality_cols = [col for col in df_filtered.select_dtypes(include=['object']).columns if df_filtered[col].nunique() > 50]
df_filtered = df_filtered.drop(columns=['SalesID', 'saledate'] + high_cardinality_cols, errors='ignore')

# Identify categorical columns
categorical_cols = df_filtered.select_dtypes(include=['object']).columns

# Apply Ordinal Encoding to 'YearMade_Bucket'
year_bucket_encoder = OrdinalEncoder(categories=[["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]])
df_filtered["YearMade_Bucket"] = year_bucket_encoder.fit_transform(df_filtered[["YearMade_Bucket"]])

# Identify high, moderate, and low cardinality categorical columns
high_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() >= 50]
moderate_cardinality_cols = [col for col in categorical_cols if 10 <= df_filtered[col].nunique() < 50]
low_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() < 10]

# Fill missing categorical values with "Unknown"
for col in categorical_cols:
    df_filtered[col] = df_filtered[col].fillna("Unknown")

# Encoding strategy

# Apply One-Hot Encoding (OHE) to low-cardinality categories
df_encoded = pd.get_dummies(df_filtered, columns=low_cardinality_cols, drop_first=True)

# Apply Ordinal Encoding to moderate-cardinality categories
if moderate_cardinality_cols:
    ord_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    df_encoded[moderate_cardinality_cols] = ord_encoder.fit_transform(df_encoded[moderate_cardinality_cols])

# Apply Frequency Encoding to high-cardinality categories
for col in high_cardinality_cols:
    freq_encoding = df_encoded[col].value_counts().to_dict()
    df_encoded[col] = df_encoded[col].map(freq_encoding)

# Drop rows with missing SalePrice (target variable)
df_encoded = df_encoded.dropna(subset=['SalePrice'])

# Split into features and target
X = df_encoded.drop(columns=['SalePrice'])
y = df_encoded['SalePrice']

# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

# Display model performance
mae, rmse

C:\Users\eitanb\AppData\Local\Temp\ipykernel_20204\3281401804.py:9: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path_train)


update_YearMade START


C:\Users\eitanb\AppData\Local\Temp\ipykernel_20204\3281401804.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1980. 1980. 1994. ... 2003. 2004. 2004.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
C:\Users\eitanb\AppData\Local\Temp\ipykernel_20204\3281401804.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['YearM

update_YearMade END


(4688.707301831825, 7572.598774518791)

In [57]:
ProductSize_path=r'C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/ProductSize_Null_fix_model.pkl'
file_path_train = r"C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/Train.csv"
df = pd.read_csv(file_path_train)

def extract_horsepower(df):
    """
    Extracts and imputes missing Horsepower values from fiProductClassDesc.
    Optimized for performance using vectorized operations.
    """
    def extract_numeric_range(value):
        """Extracts the average horsepower value from a range like '100 to 120 Horsepower'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:
            return float(numbers[0])
        return np.nan
    
    # Identify rows with Horsepower in fiProductClassDesc
    mask_horsepower = df['fiProductClassDesc'].str.contains("Horsepower", na=False)
    
    # Extract horsepower values
    df.loc[mask_horsepower, 'Horsepower_Unit_Type'] = 'Horsepower'
    df.loc[mask_horsepower, 'Extracted_Horsepower'] = df.loc[mask_horsepower, 'fiProductClassDesc'].apply(extract_numeric_range)
    
    # Handle 'Variable' and 'No' values in Engine_Horsepower
    df.loc[df['Engine_Horsepower'].isin(['Variable', 'No']), 'Engine_Horsepower'] = np.nan
    
    # Impute missing Engine_Horsepower values where Extracted_Horsepower is available
    df['Engine_Horsepower_Imputed'] = df['Engine_Horsepower'].combine_first(df['Extracted_Horsepower'])
    
    # Fill remaining NaN values with median horsepower
    #df['Engine_Horsepower_Imputed'].fillna(df['Engine_Horsepower_Imputed'].median(), inplace=True)
    df.drop(columns='Engine_Horsepower',inplace=True)
    
    return df
def preprocess_product_size(df):
    """
    Function to preprocess and impute missing ProductSize values based on fiProductClassDesc.
    
    Steps:
    1. Extract Product Type and Metric Tons / Horsepower from fiProductClassDesc.
    2. Identify Unit Type (Metric Tons, Horsepower, or Lb Operating Capacity).
    3. Convert Metric Tons / Horsepower to numerical values.
    4. Use Metric Tons to impute missing ProductSize values.
    """
    
    # Step 1: Extract Product Type and Size Descriptor
    def split_product_desc(desc):
        if pd.isna(desc):
            return pd.NA, pd.NA
        match = re.search(r'(.+?)-\s*([\d\.]+.*)', desc)
        if match:
            return match.group(1).strip(), match.group(2).strip()
        return desc.strip(), pd.NA  # If no match, return full description as product
    
    df[['Product_Type', 'Metric_Tons_HP']] = df['fiProductClassDesc'].apply(lambda x: pd.Series(split_product_desc(x)))
    
    # Step 2: Identify Unit Type
    def extract_unit_type(desc):
        if pd.isna(desc):
            return pd.NA
        if "Metric Tons" in desc:
            return "Metric Tons"
        elif "Horsepower" in desc:
            return "Horsepower"
        elif "Lb Operating Capacity" in desc:
            return "Lb Operating Capacity"
        return pd.NA
    
    df["Unit_Type"] = df["Metric_Tons_HP"].apply(extract_unit_type)
    
    # Step 3: Convert Metric Tons to numerical values
    def extract_numeric_range(value):
        """Extracts the average value from a range like '12.0 to 14.0 Metric Tons'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:  # If range exists, take the average
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:  # If only one number exists, use it
            return float(numbers[0])
        return np.nan
    
    df['Metric_Tons_Value'] = df['Metric_Tons_HP'].apply(lambda x: extract_numeric_range(x) if isinstance(x, str) else np.nan)
    
    # Step 4: Impute missing ProductSize values using Metric Tons
    def impute_product_size(row):
        if pd.isna(row['ProductSize']) and not pd.isna(row['Metric_Tons_Value']) and (row['Metric_Tons_Value']=='Metric Tons'):
            if row['Metric_Tons_Value']  <= 5:
                return 'Mini'
            elif 5 < row['Metric_Tons_Value'] <= 75:
                return 'Compact'
            elif 20 < row['Metric_Tons_Value'] <= 50:
                return 'Large / Medium'
            elif 75 < row['Metric_Tons_Value'] <= 200:
                return 'Medium'
            elif row['Metric_Tons_Value'] > 200:
                return 'Large'
        return row['ProductSize']
    
    df['ProductSize_Imputed'] = df.apply(impute_product_size, axis=1)
    df.drop(columns='ProductSize',inplace=True)
    
    return df
df1=preprocess_product_size(df)

C:\Users\eitanb\AppData\Local\Temp\ipykernel_20204\2475275052.py:3: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path_train)


In [15]:
# Re-import necessary libraries due to execution reset
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

def extract_horsepower(df):
    """
    Extracts and imputes missing Horsepower values from fiProductClassDesc.
    Optimized for performance using vectorized operations.
    """
    def extract_numeric_range(value):
        """Extracts the average horsepower value from a range like '100 to 120 Horsepower'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:
            return float(numbers[0])
        return np.nan
    
    # Identify rows with Horsepower in fiProductClassDesc
    mask_horsepower = df['fiProductClassDesc'].str.contains("Horsepower", na=False)
    
    # Extract horsepower values
    df.loc[mask_horsepower, 'Horsepower_Unit_Type'] = 'Horsepower'
    df.loc[mask_horsepower, 'Extracted_Horsepower'] = df.loc[mask_horsepower, 'fiProductClassDesc'].apply(extract_numeric_range)
    
    # Handle 'Variable' and 'No' values in Engine_Horsepower
    df.loc[df['Engine_Horsepower'].isin(['Variable', 'No']), 'Engine_Horsepower'] = np.nan
    
    # Impute missing Engine_Horsepower values where Extracted_Horsepower is available
    df['Engine_Horsepower_Imputed'] = df['Engine_Horsepower'].combine_first(df['Extracted_Horsepower'])
    
    # Fill remaining NaN values with median horsepower
    #df['Engine_Horsepower_Imputed'].fillna(df['Engine_Horsepower_Imputed'].median(), inplace=True)
    df.drop(columns='Engine_Horsepower',inplace=True)
    
    return df

def preprocess_product_size(df):
    """
    Function to preprocess and impute missing ProductSize values based on fiProductClassDesc.
    
    Steps:
    1. Extract Product Type and Metric Tons / Horsepower from fiProductClassDesc.
    2. Identify Unit Type (Metric Tons, Horsepower, or Lb Operating Capacity).
    3. Convert Metric Tons / Horsepower to numerical values.
    4. Use Metric Tons to impute missing ProductSize values.
    """
    
    # Step 1: Extract Product Type and Size Descriptor
    def split_product_desc(desc):
        if pd.isna(desc):
            return pd.NA, pd.NA
        match = re.search(r'(.+?)-\s*([\d\.]+.*)', desc)
        if match:
            return match.group(1).strip(), match.group(2).strip()
        return desc.strip(), pd.NA  # If no match, return full description as product
    
    df[['Product_Type', 'Metric_Tons_HP']] = df['fiProductClassDesc'].apply(lambda x: pd.Series(split_product_desc(x)))
    
    # Step 2: Identify Unit Type
    def extract_unit_type(desc):
        if pd.isna(desc):
            return pd.NA
        if "Metric Tons" in desc:
            return "Metric Tons"
        elif "Horsepower" in desc:
            return "Horsepower"
        elif "Lb Operating Capacity" in desc:
            return "Lb Operating Capacity"
        return pd.NA
    
    df["Unit_Type"] = df["Metric_Tons_HP"].apply(extract_unit_type)
    
    # Step 3: Convert Metric Tons to numerical values
    def extract_numeric_range(value):
        """Extracts the average value from a range like '12.0 to 14.0 Metric Tons'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:  # If range exists, take the average
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:  # If only one number exists, use it
            return float(numbers[0])
        return np.nan
    
    df['Metric_Tons_Value'] = df['Metric_Tons_HP'].apply(lambda x: extract_numeric_range(x) if isinstance(x, str) else np.nan)
    
    # Step 4: Impute missing ProductSize values using Metric Tons
    def impute_product_size(row):
        if pd.isna(row['ProductSize']) and not pd.isna(row['Metric_Tons_Value']) and (row['Metric_Tons_Value']=='Metric Tons'):
            if row['Metric_Tons_Value']  <= 5:
                return 'Mini'
            elif 5 < row['Metric_Tons_Value'] <= 75:
                return 'Compact'
            elif 20 < row['Metric_Tons_Value'] <= 50:
                return 'Large / Medium'
            elif 75 < row['Metric_Tons_Value'] <= 200:
                return 'Medium'
            elif row['Metric_Tons_Value'] > 200:
                return 'Large'
        return row['ProductSize']
    
    df['ProductSize_Imputed'] = df.apply(impute_product_size, axis=1)
    df.drop(columns='ProductSize',inplace=True)
    
    return df

# Reload the dataset
ProductSize_path=r'C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/ProductSize_Null_fix_model.pkl'
file_path_train = r"C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/Train.csv"
df = pd.read_csv(file_path_train)
#df=df[['SalesID','saledate',"Enclosure","YearMade","ModelID","MachineID","Ripper","ProductGroup","ProductSize","ProductGroupDesc","Tire_Size","MachineHoursCurrentMeter","Blade_Type","ProductGroupDesc","Travel_Controls","auctioneerID","datasource","Drive_System","Pushblock","Tire_Size","Ride_Control","UsageBand","Coupler","Engine_Horsepower_Imputed"]].copy()

# Convert 'saledate' to datetime and extract year, month, and day
df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')
df['sale_year'] = df['saledate'].dt.year
df['sale_month'] = df['saledate'].dt.month
df['sale_day'] = df['saledate'].dt.day

df=extract_horsepower(df)


df=preprocess_product_size(df)

# Define function to update 'YearMade' and create 'YearMade_Bucket'
def update_YearMade(df):
    print("update_YearMade START")
    
    # Compute median YearMade for each ModelID
    model_medians = df.loc[df['YearMade'] > 1000].groupby('ModelID')['YearMade'].median()
    
    # Update YearMade where it is 1000 using ModelID median
    df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
    
    # Compute overall median YearMade excluding 1000 values
    overall_median = df.loc[df['YearMade'] > 1000, 'YearMade'].median()
    
    # Replace any remaining 1000 values with overall median
    df['YearMade'].fillna(overall_median, inplace=True)
        
    # Bucketize YearMade
    bins = [0, 1980, 1995, 2005, 2010, 2025]
    labels = ["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]
    df['YearMade_Bucket'] = pd.cut(df['YearMade'], bins=bins, labels=labels, right=True)
    
    print("update_YearMade END")
    return df

# Apply YearMade update
df = update_YearMade(df)

# Filter dataset to only include the last 5 years
recent_years = df['sale_year'].dropna().unique()
recent_years.sort()
#selected_years = recent_years[-5:]
#df_filtered = df[df['sale_year'].isin(selected_years)]

# Drop irrelevant columns: 'SalesID', 'saledate'
high_cardinality_cols = [col for col in df_filtered.select_dtypes(include=['object']).columns if df_filtered[col].nunique() > 50]
df_filtered = df_filtered.drop(columns=['SalesID', 'saledate'] + high_cardinality_cols, errors='ignore')

# Identify categorical columns
categorical_cols = df_filtered.select_dtypes(include=['object']).columns

df_filtered["YearMade_Bucket"] = df_filtered["YearMade_Bucket"].astype(str).fillna("Unknown")

# Define category order and apply Ordinal Encoding
year_bucket_encoder = OrdinalEncoder(
    categories=[["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025", "Unknown"]],
    handle_unknown="use_encoded_value",
    unknown_value=-1
)

df_filtered["YearMade_Bucket"] = year_bucket_encoder.fit_transform(df_filtered[["YearMade_Bucket"]])
# Apply Ordinal Encoding to 'YearMade_Bucket'
#year_bucket_encoder = OrdinalEncoder(categories=[["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]])
#df_filtered["YearMade_Bucket"] = year_bucket_encoder.fit_transform(df_filtered[["YearMade_Bucket"]])

# Identify high, moderate, and low cardinality categorical columns
high_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() >= 50]
moderate_cardinality_cols = [col for col in categorical_cols if 10 <= df_filtered[col].nunique() < 50]
low_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() < 10]

# Fill missing categorical values with "Unknown"
for col in categorical_cols:
    df_filtered[col] = df_filtered[col].fillna("Unknown")

# Encoding strategy

# Apply One-Hot Encoding (OHE) to low-cardinality categories
df_encoded = pd.get_dummies(df_filtered, columns=low_cardinality_cols, drop_first=True)

# Apply Ordinal Encoding to moderate-cardinality categories
if moderate_cardinality_cols:
    ord_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    df_encoded[moderate_cardinality_cols] = ord_encoder.fit_transform(df_encoded[moderate_cardinality_cols])

# Apply Frequency Encoding to high-cardinality categories
for col in high_cardinality_cols:
    freq_encoding = df_encoded[col].value_counts().to_dict()
    df_encoded[col] = df_encoded[col].map(freq_encoding)

# Drop rows with missing SalePrice (target variable)
df_encoded = df_encoded.dropna(subset=['SalePrice'])

# Split into features and target
X = df_encoded.drop(columns=['SalePrice'])
y = df_encoded['SalePrice']

# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

# Display model performance
mae, rmse

C:\Users\eitanb\AppData\Local\Temp\ipykernel_18780\4042277343.py:113: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path_train)
C:\Users\eitanb\AppData\Local\Temp\ipykernel_18780\4042277343.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df['Engine_Horsepower_Imputed'] = df['Engine_Horsepower'].combine_first(df['Extracted_Horsepower'])


update_YearMade START


C:\Users\eitanb\AppData\Local\Temp\ipykernel_18780\4042277343.py:135: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1980. 1980. 1994. ... 2003. 2004. 2004.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
C:\Users\eitanb\AppData\Local\Temp\ipykernel_18780\4042277343.py:141: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Yea

update_YearMade END


(4690.262789670813, 7587.100811299758)

In [3]:
# Re-import necessary libraries due to execution reset
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

def extract_horsepower(df):
    """
    Extracts and imputes missing Horsepower values from fiProductClassDesc.
    Optimized for performance using vectorized operations.
    """
    def extract_numeric_range(value):
        """Extracts the average horsepower value from a range like '100 to 120 Horsepower'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:
            return float(numbers[0])
        return np.nan
    
    # Identify rows with Horsepower in fiProductClassDesc
    mask_horsepower = df['fiProductClassDesc'].str.contains("Horsepower", na=False)
    
    # Extract horsepower values
    df.loc[mask_horsepower, 'Horsepower_Unit_Type'] = 'Horsepower'
    df.loc[mask_horsepower, 'Extracted_Horsepower'] = df.loc[mask_horsepower, 'fiProductClassDesc'].apply(extract_numeric_range)
    
    # Handle 'Variable' and 'No' values in Engine_Horsepower
    df.loc[df['Engine_Horsepower'].isin(['Variable', 'No']), 'Engine_Horsepower'] = np.nan
    
    # Impute missing Engine_Horsepower values where Extracted_Horsepower is available
    df['Engine_Horsepower_Imputed'] = df['Engine_Horsepower'].combine_first(df['Extracted_Horsepower'])
    
    # Fill remaining NaN values with median horsepower
    #df['Engine_Horsepower_Imputed'].fillna(df['Engine_Horsepower_Imputed'].median(), inplace=True)
    df.drop(columns='Engine_Horsepower',inplace=True)
    
    return df

def preprocess_product_size(df):
    """
    Function to preprocess and impute missing ProductSize values based on fiProductClassDesc.
    
    Steps:
    1. Extract Product Type and Metric Tons / Horsepower from fiProductClassDesc.
    2. Identify Unit Type (Metric Tons, Horsepower, or Lb Operating Capacity).
    3. Convert Metric Tons / Horsepower to numerical values.
    4. Use Metric Tons to impute missing ProductSize values.
    """
    
    # Step 1: Extract Product Type and Size Descriptor
    def split_product_desc(desc):
        if pd.isna(desc):
            return pd.NA, pd.NA
        match = re.search(r'(.+?)-\s*([\d\.]+.*)', desc)
        if match:
            return match.group(1).strip(), match.group(2).strip()
        return desc.strip(), pd.NA  # If no match, return full description as product
    
    df[['Product_Type', 'Metric_Tons_HP']] = df['fiProductClassDesc'].apply(lambda x: pd.Series(split_product_desc(x)))
    
    # Step 2: Identify Unit Type
    def extract_unit_type(desc):
        if pd.isna(desc):
            return pd.NA
        if "Metric Tons" in desc:
            return "Metric Tons"
        elif "Horsepower" in desc:
            return "Horsepower"
        elif "Lb Operating Capacity" in desc:
            return "Lb Operating Capacity"
        return pd.NA
    
    df["Unit_Type"] = df["Metric_Tons_HP"].apply(extract_unit_type)
    
    # Step 3: Convert Metric Tons to numerical values
    def extract_numeric_range(value):
        """Extracts the average value from a range like '12.0 to 14.0 Metric Tons'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:  # If range exists, take the average
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:  # If only one number exists, use it
            return float(numbers[0])
        return np.nan
    
    df['Metric_Tons_Value'] = df['Metric_Tons_HP'].apply(lambda x: extract_numeric_range(x) if isinstance(x, str) else np.nan)
    
    # Step 4: Impute missing ProductSize values using Metric Tons
    def impute_product_size(row):
        if pd.isna(row['ProductSize']) and not pd.isna(row['Metric_Tons_Value']) and (row['Metric_Tons_Value']=='Metric Tons'):
            if row['Metric_Tons_Value']  <= 5:
                return 'Mini'
            elif 5 < row['Metric_Tons_Value'] <= 75:
                return 'Compact'
            elif 20 < row['Metric_Tons_Value'] <= 50:
                return 'Large / Medium'
            elif 75 < row['Metric_Tons_Value'] <= 200:
                return 'Medium'
            elif row['Metric_Tons_Value'] > 200:
                return 'Large'
        return row['ProductSize']
    
    df['ProductSize_Imputed'] = df.apply(impute_product_size, axis=1)
    df.drop(columns=['ProductSize','Metric_Tons_Value','Metric_Tons_HP','Unit_Type'],inplace=True)

    
    
    return df
def Enclosure_fun(df):
    list1=['EROPS','OROPS','EROPS AC']
    list2=['NO ROPS','None or Unspecified']
    df['Enclosure_cat']=0
    df.loc[df['Enclosure'].isin(list1), 'Enclosure_cat'] = 1
    df.loc[df['Enclosure'].isin(list2), 'Enclosure_cat'] = 0
    df.loc[df['Enclosure']=='EROPS w AC','Enclosure_cat']=2
    df.drop(columns='Enclosure',inplace=True)
    return df
# Reload the dataset
ProductSize_path=r'C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/ProductSize_Null_fix_model.pkl'
file_path_train = r"C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/Train.csv"
df = pd.read_csv(file_path_train)
#df=df[['SalesID','saledate',"Enclosure","YearMade","ModelID","MachineID","Ripper","ProductGroup","ProductSize","ProductGroupDesc","Tire_Size","MachineHoursCurrentMeter","Blade_Type","ProductGroupDesc","Travel_Controls","auctioneerID","datasource","Drive_System","Pushblock","Tire_Size","Ride_Control","UsageBand","Coupler","Engine_Horsepower_Imputed"]].copy()

# Convert 'saledate' to datetime and extract year, month, and day
df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')
df['sale_year'] = df['saledate'].dt.year
df['sale_month'] = df['saledate'].dt.month
df['sale_day'] = df['saledate'].dt.day
df=Enclosure_fun(df)
df=extract_horsepower(df)


df=preprocess_product_size(df)

# Define function to update 'YearMade' and create 'YearMade_Bucket'
def update_YearMade(df):
    print("update_YearMade START")
    
    # Compute median YearMade for each ModelID
    model_medians = df.loc[df['YearMade'] > 1000].groupby('ModelID')['YearMade'].median()
    
    # Update YearMade where it is 1000 using ModelID median
    df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
    
    # Compute overall median YearMade excluding 1000 values
    overall_median = df.loc[df['YearMade'] > 1000, 'YearMade'].median()
    
    # Replace any remaining 1000 values with overall median
    df['YearMade'].fillna(overall_median, inplace=True)
        
    # Bucketize YearMade
    bins = [0, 1980, 1995, 2005, 2010, 2025]
    labels = ["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]
    df['YearMade_Bucket'] = pd.cut(df['YearMade'], bins=bins, labels=labels, right=True)
    
    print("update_YearMade END")
    return df

# Apply YearMade update
df = update_YearMade(df)

df.drop(columns='fiProductClassDesc',inplace=True)

# Filter dataset to only include the last 5 years
recent_years = df['sale_year'].dropna().unique()
recent_years.sort()
selected_years = recent_years[-5:]
df_filtered = df[df['sale_year'].isin(selected_years)]

# Drop irrelevant columns: 'SalesID', 'saledate'
high_cardinality_cols = [col for col in df_filtered.select_dtypes(include=['object']).columns if df_filtered[col].nunique() > 50]
df_filtered = df_filtered.drop(columns=['SalesID', 'saledate'] + high_cardinality_cols, errors='ignore')

# Identify categorical columns
categorical_cols = df_filtered.select_dtypes(include=['object']).columns

df_filtered["YearMade_Bucket"] = df_filtered["YearMade_Bucket"].astype(str).fillna("Unknown")

# Define category order and apply Ordinal Encoding
year_bucket_encoder = OrdinalEncoder(
    categories=[["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025", "Unknown"]],
    handle_unknown="use_encoded_value",
    unknown_value=-1
)

df_filtered["YearMade_Bucket"] = year_bucket_encoder.fit_transform(df_filtered[["YearMade_Bucket"]])
# Apply Ordinal Encoding to 'YearMade_Bucket'
#year_bucket_encoder = OrdinalEncoder(categories=[["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]])
#df_filtered["YearMade_Bucket"] = year_bucket_encoder.fit_transform(df_filtered[["YearMade_Bucket"]])

# Identify high, moderate, and low cardinality categorical columns
high_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() >= 50]
moderate_cardinality_cols = [col for col in categorical_cols if 10 <= df_filtered[col].nunique() < 50]
low_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() < 10]

# Fill missing categorical values with "Unknown"
for col in categorical_cols:
    df_filtered[col] = df_filtered[col].fillna("Unknown")

# Encoding strategy

# Apply One-Hot Encoding (OHE) to low-cardinality categories
df_encoded = pd.get_dummies(df_filtered, columns=low_cardinality_cols, drop_first=True)

# Apply Ordinal Encoding to moderate-cardinality categories
if moderate_cardinality_cols:
    ord_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    df_encoded[moderate_cardinality_cols] = ord_encoder.fit_transform(df_encoded[moderate_cardinality_cols])

# Apply Frequency Encoding to high-cardinality categories
for col in high_cardinality_cols:
    freq_encoding = df_encoded[col].value_counts().to_dict()
    df_encoded[col] = df_encoded[col].map(freq_encoding)

# Drop rows with missing SalePrice (target variable)
df_encoded = df_encoded.dropna(subset=['SalePrice'])

# Split into features and target
X = df_encoded.drop(columns=['SalePrice'])
y = df_encoded['SalePrice']

# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

# Display model performance
mae, rmse

C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\1994876309.py:123: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path_train)
C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\1994876309.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df['Engine_Horsepower_Imputed'] = df['Engine_Horsepower'].combine_first(df['Extracted_Horsepower'])


update_YearMade START


C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\1994876309.py:145: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1980. 1980. 1994. ... 2003. 2004. 2004.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\1994876309.py:151: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Yea

update_YearMade END


(4833.026361105839, 7785.955971072109)

In [4]:
# Optional: Log feature importances
importances = rf_model.feature_importances_

# Create a zip of feature names and feature importances
feature_zip = zip(rf_model.feature_names_in_, rf_model.feature_importances_)

# Sort the zip by feature importance in descending order
sorted_feature_zip = sorted(feature_zip, key=lambda x: x[1], reverse=True)

# Print the sorted feature names, importances, and index
for idx, (feat, importance) in enumerate(sorted_feature_zip, 1):
  print({f"feature_{idx}_{feat}_importance" : importance})


{'feature_1_Enclosure_cat_importance': 0.28921846791011036}
{'feature_2_YearMade_importance': 0.17637686207943865}
{'feature_3_ModelID_importance': 0.10128183415154628}
{'feature_4_Engine_Horsepower_Imputed_importance': 0.07265473670191194}
{'feature_5_Extracted_Horsepower_importance': 0.06481618771286045}
{'feature_6_ProductSize_Imputed_Large / Medium_importance': 0.05829905999977042}
{'feature_7_sale_year_importance': 0.03754676387293485}
{'feature_8_MachineID_importance': 0.03002420773810382}
{'feature_9_sale_month_importance': 0.017387648234200925}
{'feature_10_sale_day_importance': 0.015795638656916917}
{'feature_11_MachineHoursCurrentMeter_importance': 0.012463876084173646}
{'feature_12_ProductSize_Imputed_Mini_importance': 0.012124578230267354}
{'feature_13_ProductGroup_SSL_importance': 0.010411929293020669}
{'feature_14_ProductGroupDesc_Skid Steer Loaders_importance': 0.00861604672765212}
{'feature_15_ProductSize_Imputed_Small_importance': 0.008268458145916589}
{'feature_16_auc

In [5]:
# Re-import necessary libraries due to execution reset
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

def extract_horsepower(df):
    """
    Extracts and imputes missing Horsepower values from fiProductClassDesc.
    Optimized for performance using vectorized operations.
    """
    def extract_numeric_range(value):
        """Extracts the average horsepower value from a range like '100 to 120 Horsepower'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:
            return float(numbers[0])
        return np.nan
    
    # Identify rows with Horsepower in fiProductClassDesc
    mask_horsepower = df['fiProductClassDesc'].str.contains("Horsepower", na=False)
    
    # Extract horsepower values
    df.loc[mask_horsepower, 'Horsepower_Unit_Type'] = 'Horsepower'
    df.loc[mask_horsepower, 'Extracted_Horsepower'] = df.loc[mask_horsepower, 'fiProductClassDesc'].apply(extract_numeric_range)
    
    # Handle 'Variable' and 'No' values in Engine_Horsepower
    df.loc[df['Engine_Horsepower'].isin(['Variable', 'No']), 'Engine_Horsepower'] = np.nan
    
    # Impute missing Engine_Horsepower values where Extracted_Horsepower is available
    df['Engine_Horsepower_Imputed'] = df['Engine_Horsepower'].combine_first(df['Extracted_Horsepower'])
    
    # Fill remaining NaN values with median horsepower
    #df['Engine_Horsepower_Imputed'].fillna(df['Engine_Horsepower_Imputed'].median(), inplace=True)
    df.drop(columns='Engine_Horsepower',inplace=True)
    
    return df

def preprocess_product_size(df):
    """
    Function to preprocess and impute missing ProductSize values based on fiProductClassDesc.
    
    Steps:
    1. Extract Product Type and Metric Tons / Horsepower from fiProductClassDesc.
    2. Identify Unit Type (Metric Tons, Horsepower, or Lb Operating Capacity).
    3. Convert Metric Tons / Horsepower to numerical values.
    4. Use Metric Tons to impute missing ProductSize values.
    """
    
    # Step 1: Extract Product Type and Size Descriptor
    def split_product_desc(desc):
        if pd.isna(desc):
            return pd.NA, pd.NA
        match = re.search(r'(.+?)-\s*([\d\.]+.*)', desc)
        if match:
            return match.group(1).strip(), match.group(2).strip()
        return desc.strip(), pd.NA  # If no match, return full description as product
    
    df[['Product_Type', 'Metric_Tons_HP']] = df['fiProductClassDesc'].apply(lambda x: pd.Series(split_product_desc(x)))
    
    # Step 2: Identify Unit Type
    def extract_unit_type(desc):
        if pd.isna(desc):
            return pd.NA
        if "Metric Tons" in desc:
            return "Metric Tons"
        elif "Horsepower" in desc:
            return "Horsepower"
        elif "Lb Operating Capacity" in desc:
            return "Lb Operating Capacity"
        return pd.NA
    
    df["Unit_Type"] = df["Metric_Tons_HP"].apply(extract_unit_type)
    
    # Step 3: Convert Metric Tons to numerical values
    def extract_numeric_range(value):
        """Extracts the average value from a range like '12.0 to 14.0 Metric Tons'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:  # If range exists, take the average
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:  # If only one number exists, use it
            return float(numbers[0])
        return np.nan
    
    df['Metric_Tons_Value'] = df['Metric_Tons_HP'].apply(lambda x: extract_numeric_range(x) if isinstance(x, str) else np.nan)
    
    # Step 4: Impute missing ProductSize values using Metric Tons
    def impute_product_size(row):
        if pd.isna(row['ProductSize']) and not pd.isna(row['Metric_Tons_Value']) and (row['Metric_Tons_Value']=='Metric Tons'):
            if row['Metric_Tons_Value']  <= 5:
                return 'Mini'
            elif 5 < row['Metric_Tons_Value'] <= 75:
                return 'Compact'
            elif 20 < row['Metric_Tons_Value'] <= 50:
                return 'Large / Medium'
            elif 75 < row['Metric_Tons_Value'] <= 200:
                return 'Medium'
            elif row['Metric_Tons_Value'] > 200:
                return 'Large'
        return row['ProductSize']
    
    df['ProductSize_Imputed'] = df.apply(impute_product_size, axis=1)
    df.drop(columns=['ProductSize','Metric_Tons_Value','Metric_Tons_HP','Unit_Type'],inplace=True)

    
    
    return df
def Enclosure_fun(df):
    list1=['EROPS','OROPS','EROPS AC']
    list2=['NO ROPS','None or Unspecified']
    df['Enclosure_cat']=0
    df.loc[df['Enclosure'].isin(list1), 'Enclosure_cat'] = 1
    df.loc[df['Enclosure'].isin(list2), 'Enclosure_cat'] = 0
    df.loc[df['Enclosure']=='EROPS w AC','Enclosure_cat']=2
    df.drop(columns='Enclosure',inplace=True)
    return df
# Define categories based on observed price trends
def categorize_hydraulics(value):
    if value in ["Missing", "Auxiliary", "Standard"]:
        return "Basic"
    elif value in ["2 Valve", "None or Unspecified", "3 Valve"]:
        return "Mid-Level"
    else:
        return "Advanced"

# Apply categorization
df["Hydraulics_Category"] = df["Hydraulics"].apply(categorize_hydraulics)

# Reload the dataset
ProductSize_path=r'C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/ProductSize_Null_fix_model.pkl'
file_path_train = r"C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/Train.csv"
df = pd.read_csv(file_path_train)
#df=df[['SalesID','saledate',"Enclosure","YearMade","ModelID","MachineID","Ripper","ProductGroup","ProductSize","ProductGroupDesc","Tire_Size","MachineHoursCurrentMeter","Blade_Type","ProductGroupDesc","Travel_Controls","auctioneerID","datasource","Drive_System","Pushblock","Tire_Size","Ride_Control","UsageBand","Coupler","Engine_Horsepower_Imputed"]].copy()

# Convert 'saledate' to datetime and extract year, month, and day
df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')
df['sale_year'] = df['saledate'].dt.year
df['sale_month'] = df['saledate'].dt.month
df['sale_day'] = df['saledate'].dt.day

# Apply categorization
df["Hydraulics_Category"] = df["Hydraulics"].apply(categorize_hydraulics)

df=Enclosure_fun(df)
df=extract_horsepower(df)


df=preprocess_product_size(df)

# Define function to update 'YearMade' and create 'YearMade_Bucket'
def update_YearMade(df):
    print("update_YearMade START")
    
    # Compute median YearMade for each ModelID
    model_medians = df.loc[df['YearMade'] > 1000].groupby('ModelID')['YearMade'].median()
    
    # Update YearMade where it is 1000 using ModelID median
    df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
    
    # Compute overall median YearMade excluding 1000 values
    overall_median = df.loc[df['YearMade'] > 1000, 'YearMade'].median()
    
    # Replace any remaining 1000 values with overall median
    df['YearMade'].fillna(overall_median, inplace=True)
        
    # Bucketize YearMade
    bins = [0, 1980, 1995, 2005, 2010, 2025]
    labels = ["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]
    df['YearMade_Bucket'] = pd.cut(df['YearMade'], bins=bins, labels=labels, right=True)
    
    print("update_YearMade END")
    return df

# Apply YearMade update
df = update_YearMade(df)

df.drop(columns=['fiProductClassDesc','Hydraulics'],inplace=True)


# Filter dataset to only include the last 5 years
recent_years = df['sale_year'].dropna().unique()
recent_years.sort()
selected_years = recent_years[-5:]
df_filtered = df[df['sale_year'].isin(selected_years)]

# Drop irrelevant columns: 'SalesID', 'saledate'
high_cardinality_cols = [col for col in df_filtered.select_dtypes(include=['object']).columns if df_filtered[col].nunique() > 50]
df_filtered = df_filtered.drop(columns=['SalesID', 'saledate'] + high_cardinality_cols, errors='ignore')

# Identify categorical columns
categorical_cols = df_filtered.select_dtypes(include=['object']).columns

df_filtered["YearMade_Bucket"] = df_filtered["YearMade_Bucket"].astype(str).fillna("Unknown")

# Define category order and apply Ordinal Encoding
year_bucket_encoder = OrdinalEncoder(
    categories=[["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025", "Unknown"]],
    handle_unknown="use_encoded_value",
    unknown_value=-1
)

df_filtered["YearMade_Bucket"] = year_bucket_encoder.fit_transform(df_filtered[["YearMade_Bucket"]])
# Apply Ordinal Encoding to 'YearMade_Bucket'
#year_bucket_encoder = OrdinalEncoder(categories=[["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]])
#df_filtered["YearMade_Bucket"] = year_bucket_encoder.fit_transform(df_filtered[["YearMade_Bucket"]])

# Identify high, moderate, and low cardinality categorical columns
high_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() >= 50]
moderate_cardinality_cols = [col for col in categorical_cols if 10 <= df_filtered[col].nunique() < 50]
low_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() < 10]

# Fill missing categorical values with "Unknown"
for col in categorical_cols:
    df_filtered[col] = df_filtered[col].fillna("Unknown")

# Encoding strategy

# Apply One-Hot Encoding (OHE) to low-cardinality categories
df_encoded = pd.get_dummies(df_filtered, columns=low_cardinality_cols, drop_first=True)

# Apply Ordinal Encoding to moderate-cardinality categories
if moderate_cardinality_cols:
    ord_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    df_encoded[moderate_cardinality_cols] = ord_encoder.fit_transform(df_encoded[moderate_cardinality_cols])

# Apply Frequency Encoding to high-cardinality categories
for col in high_cardinality_cols:
    freq_encoding = df_encoded[col].value_counts().to_dict()
    df_encoded[col] = df_encoded[col].map(freq_encoding)

# Drop rows with missing SalePrice (target variable)
df_encoded = df_encoded.dropna(subset=['SalePrice'])

# Split into features and target
X = df_encoded.drop(columns=['SalePrice'])
y = df_encoded['SalePrice']

# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

# Display model performance
mae, rmse

C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\1407107489.py:135: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path_train)
C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\1407107489.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df['Engine_Horsepower_Imputed'] = df['Engine_Horsepower'].combine_first(df['Extracted_Horsepower'])


update_YearMade START


C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\1407107489.py:161: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1980. 1980. 1994. ... 2003. 2004. 2004.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\1407107489.py:167: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Yea

update_YearMade END


(4833.200946416145, 7782.836218132495)

In [6]:
# Optional: Log feature importances
importances = rf_model.feature_importances_

# Create a zip of feature names and feature importances
feature_zip = zip(rf_model.feature_names_in_, rf_model.feature_importances_)

# Sort the zip by feature importance in descending order
sorted_feature_zip = sorted(feature_zip, key=lambda x: x[1], reverse=True)

# Print the sorted feature names, importances, and index
for idx, (feat, importance) in enumerate(sorted_feature_zip, 1):
  print({f"feature_{idx}_{feat}_importance" : importance})

{'feature_1_Enclosure_cat_importance': 0.28921994640432164}
{'feature_2_YearMade_importance': 0.17652927949578853}
{'feature_3_ModelID_importance': 0.1015162105349354}
{'feature_4_Extracted_Horsepower_importance': 0.07111184812668449}
{'feature_5_Engine_Horsepower_Imputed_importance': 0.06632425226911177}
{'feature_6_ProductSize_Imputed_Large / Medium_importance': 0.05829994934360587}
{'feature_7_sale_year_importance': 0.037558305778334175}
{'feature_8_MachineID_importance': 0.030305171386477717}
{'feature_9_sale_month_importance': 0.017495608912622744}
{'feature_10_sale_day_importance': 0.0160042991561733}
{'feature_11_MachineHoursCurrentMeter_importance': 0.012586992701690608}
{'feature_12_ProductSize_Imputed_Mini_importance': 0.01212976145386422}
{'feature_13_ProductGroupDesc_Skid Steer Loaders_importance': 0.009914426648220755}
{'feature_14_ProductGroup_SSL_importance': 0.00911290226458472}
{'feature_15_ProductSize_Imputed_Small_importance': 0.008259533608617971}
{'feature_16_aucti

In [11]:
# Re-import necessary libraries due to execution reset
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

# Define price-based categories for ProductGroup
def categorize_product_group(price):
    if price < 20000:
        return "Low Price Group"
    elif 20000 <= price < 40000:
        return "Mid Price Group"
    else:
        return "High Price Group"


def create_model_category_mapping(training_df):
    """
    Creates a mapping of ModelID to Model_Category based on fiModelDesc and SalePrice in training data.
    
    Parameters:
    training_df (pd.DataFrame): Training dataset containing fiModelDesc and SalePrice.
    
    Returns:
    dict: A dictionary mapping ModelID to Model_Category.
    """
    # Compute average SalePrice per fiModelDesc in training data
    model_avg_price = training_df.groupby("fiModelDesc")["SalePrice"].mean()
    
    # Define price categories
    def categorize_price(price):
        if price < 20000:
            return "Low Price Models"
        elif 20000 <= price < 60000:
            return "Mid Price Models"
        else:
            return "High Price Models"
    
    # Map fiModelDesc to categories
    model_category_mapping = model_avg_price.apply(categorize_price).to_dict()
    
    # Create ModelID to category mapping using fiModelDesc
    training_df["Model_Category"] = training_df["fiModelDesc"].map(model_category_mapping)
    modelid_to_category = training_df.set_index("ModelID")["Model_Category"].to_dict()
    
    return modelid_to_category, model_category_mapping

def categorize_model_id(df, modelid_to_category, model_category_mapping):
    """
    Categorizes ModelID based on precomputed price categories.
    If ModelID is not found, falls back to fiModelDesc categorization.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing ModelID and fiModelDesc columns.
    modelid_to_category (dict): Mapping of ModelID to Model_Category.
    model_category_mapping (dict): Mapping of fiModelDesc to Model_Category for fallback.
    
    Returns:
    pd.Series: A Series with categorized model price labels.
    """
    df["Predicted_Model_Category"] = df["ModelID"].map(modelid_to_category)
    
    # Handle missing ModelID by checking fiModelDesc mapping
    missing_mask = df["Predicted_Model_Category"].isna()
    df.loc[missing_mask, "Predicted_Model_Category"] = df.loc[missing_mask, "fiModelDesc"].map(model_category_mapping)
    
    # Handle any remaining missing values by assigning 'Unknown'
    df["Predicted_Model_Category"].fillna("Unknown", inplace=True)
    
    return df["Predicted_Model_Category"]



def extract_horsepower(df):
    """
    Extracts and imputes missing Horsepower values from fiProductClassDesc.
    Optimized for performance using vectorized operations.
    """
    def extract_numeric_range(value):
        """Extracts the average horsepower value from a range like '100 to 120 Horsepower'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:
            return float(numbers[0])
        return np.nan
    
    # Identify rows with Horsepower in fiProductClassDesc
    mask_horsepower = df['fiProductClassDesc'].str.contains("Horsepower", na=False)
    
    # Extract horsepower values
    df.loc[mask_horsepower, 'Horsepower_Unit_Type'] = 'Horsepower'
    df.loc[mask_horsepower, 'Extracted_Horsepower'] = df.loc[mask_horsepower, 'fiProductClassDesc'].apply(extract_numeric_range)
    
    # Handle 'Variable' and 'No' values in Engine_Horsepower
    df.loc[df['Engine_Horsepower'].isin(['Variable', 'No']), 'Engine_Horsepower'] = np.nan
    
    # Impute missing Engine_Horsepower values where Extracted_Horsepower is available
    df['Engine_Horsepower_Imputed'] = df['Engine_Horsepower'].combine_first(df['Extracted_Horsepower'])
    
    # Fill remaining NaN values with median horsepower
    #df['Engine_Horsepower_Imputed'].fillna(df['Engine_Horsepower_Imputed'].median(), inplace=True)
    df.drop(columns='Engine_Horsepower',inplace=True)
    
    return df

def preprocess_product_size(df):
    """
    Function to preprocess and impute missing ProductSize values based on fiProductClassDesc.
    
    Steps:
    1. Extract Product Type and Metric Tons / Horsepower from fiProductClassDesc.
    2. Identify Unit Type (Metric Tons, Horsepower, or Lb Operating Capacity).
    3. Convert Metric Tons / Horsepower to numerical values.
    4. Use Metric Tons to impute missing ProductSize values.
    """
    
    # Step 1: Extract Product Type and Size Descriptor
    def split_product_desc(desc):
        if pd.isna(desc):
            return pd.NA, pd.NA
        match = re.search(r'(.+?)-\s*([\d\.]+.*)', desc)
        if match:
            return match.group(1).strip(), match.group(2).strip()
        return desc.strip(), pd.NA  # If no match, return full description as product
    
    df[['Product_Type', 'Metric_Tons_HP']] = df['fiProductClassDesc'].apply(lambda x: pd.Series(split_product_desc(x)))
    
    # Step 2: Identify Unit Type
    def extract_unit_type(desc):
        if pd.isna(desc):
            return pd.NA
        if "Metric Tons" in desc:
            return "Metric Tons"
        elif "Horsepower" in desc:
            return "Horsepower"
        elif "Lb Operating Capacity" in desc:
            return "Lb Operating Capacity"
        return pd.NA
    
    df["Unit_Type"] = df["Metric_Tons_HP"].apply(extract_unit_type)
    
    # Step 3: Convert Metric Tons to numerical values
    def extract_numeric_range(value):
        """Extracts the average value from a range like '12.0 to 14.0 Metric Tons'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:  # If range exists, take the average
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:  # If only one number exists, use it
            return float(numbers[0])
        return np.nan
    
    df['Metric_Tons_Value'] = df['Metric_Tons_HP'].apply(lambda x: extract_numeric_range(x) if isinstance(x, str) else np.nan)
    
    # Step 4: Impute missing ProductSize values using Metric Tons
    def impute_product_size(row):
        if pd.isna(row['ProductSize']) and not pd.isna(row['Metric_Tons_Value']) and (row['Metric_Tons_Value']=='Metric Tons'):
            if row['Metric_Tons_Value']  <= 5:
                return 'Mini'
            elif 5 < row['Metric_Tons_Value'] <= 75:
                return 'Compact'
            elif 20 < row['Metric_Tons_Value'] <= 50:
                return 'Large / Medium'
            elif 75 < row['Metric_Tons_Value'] <= 200:
                return 'Medium'
            elif row['Metric_Tons_Value'] > 200:
                return 'Large'
        return row['ProductSize']
    
    df['ProductSize_Imputed'] = df.apply(impute_product_size, axis=1)
    df.drop(columns=['ProductSize','Metric_Tons_Value','Metric_Tons_HP','Unit_Type'],inplace=True)

    
    
    return df
def Enclosure_fun(df):
    list1=['EROPS','OROPS','EROPS AC']
    list2=['NO ROPS','None or Unspecified']
    df['Enclosure_cat']=0
    df.loc[df['Enclosure'].isin(list1), 'Enclosure_cat'] = 1
    df.loc[df['Enclosure'].isin(list2), 'Enclosure_cat'] = 0
    df.loc[df['Enclosure']=='EROPS w AC','Enclosure_cat']=2
    df.drop(columns='Enclosure',inplace=True)
    return df
# Define categories based on observed price trends
def categorize_hydraulics(value):
    if value in ["Missing", "Auxiliary", "Standard"]:
        return "Basic"
    elif value in ["2 Valve", "None or Unspecified", "3 Valve"]:
        return "Mid-Level"
    else:
        return "Advanced"



# Reload the dataset
ProductSize_path=r'C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/ProductSize_Null_fix_model.pkl'
file_path_train = r"C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/Train.csv"
df = pd.read_csv(file_path_train)
#df=df[['SalesID','saledate',"Enclosure","YearMade","ModelID","MachineID","Ripper","ProductGroup","ProductSize","ProductGroupDesc","Tire_Size","MachineHoursCurrentMeter","Blade_Type","ProductGroupDesc","Travel_Controls","auctioneerID","datasource","Drive_System","Pushblock","Tire_Size","Ride_Control","UsageBand","Coupler","Engine_Horsepower_Imputed"]].copy()

# Convert 'saledate' to datetime and extract year, month, and day
df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')
df['sale_year'] = df['saledate'].dt.year
df['sale_month'] = df['saledate'].dt.month
df['sale_day'] = df['saledate'].dt.day

# Apply categorization
df["Hydraulics_Category"] = df["Hydraulics"].apply(categorize_hydraulics)

df=Enclosure_fun(df)
df=extract_horsepower(df)

# Create mappings from training data only in saleprice 
modelid_to_category, model_category_mapping = create_model_category_mapping(df) # need to upload the modelid_to_category, model_category_mapping in the test

# Apply categorization to test data
df["Predicted_Model_Category"] = categorize_model_id(df, modelid_to_category, model_category_mapping) # need to upload the modelid_to_category, model_category_mapping in the test

# Compute the average price for each ProductGroup

product_group_avg_price = df.groupby("ProductGroup")["SalePrice"].mean()

# Map each ProductGroup to a price category
df["ProductGroup_Category"] = df["ProductGroup"].map(product_group_avg_price).apply(categorize_product_group)# need to upload the categorize_product_group in the test
df=preprocess_product_size(df)

# Define function to update 'YearMade' and create 'YearMade_Bucket'
def update_YearMade(df):
    print("update_YearMade START")
    
    # Compute median YearMade for each ModelID
    model_medians = df.loc[df['YearMade'] > 1000].groupby('ModelID')['YearMade'].median()
    
    # Update YearMade where it is 1000 using ModelID median
    df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
    
    # Compute overall median YearMade excluding 1000 values
    overall_median = df.loc[df['YearMade'] > 1000, 'YearMade'].median()
    
    # Replace any remaining 1000 values with overall median
    df['YearMade'].fillna(overall_median, inplace=True)
        
    # Bucketize YearMade
    bins = [0, 1980, 1995, 2005, 2010, 2025]
    labels = ["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]
    df['YearMade_Bucket'] = pd.cut(df['YearMade'], bins=bins, labels=labels, right=True)
    
    print("update_YearMade END")
    return df

# Apply YearMade update
df = update_YearMade(df)

#df.drop(columns=['fiProductClassDesc','Hydraulics','fiSecondaryDesc','auctioneerID','fiModelDesc','fiBaseModel','ProductGroupDesc','MachineID','ProductGroup'],inplace=True)


# Filter dataset to only include the last 5 years
recent_years = df['sale_year'].dropna().unique()
recent_years.sort()
selected_years = recent_years[-5:]
df_filtered = df[df['sale_year'].isin(selected_years)]

# Drop irrelevant columns: 'SalesID', 'saledate'
high_cardinality_cols = [col for col in df_filtered.select_dtypes(include=['object']).columns if df_filtered[col].nunique() > 50]
df_filtered = df_filtered.drop(columns=['SalesID', 'saledate'] + high_cardinality_cols, errors='ignore')

# Identify categorical columns
categorical_cols = df_filtered.select_dtypes(include=['object']).columns

df_filtered["YearMade_Bucket"] = df_filtered["YearMade_Bucket"].astype(str).fillna("Unknown")

# Define category order and apply Ordinal Encoding
year_bucket_encoder = OrdinalEncoder(
    categories=[["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025", "Unknown"]],
    handle_unknown="use_encoded_value",
    unknown_value=-1
)

df_filtered["YearMade_Bucket"] = year_bucket_encoder.fit_transform(df_filtered[["YearMade_Bucket"]])
# Apply Ordinal Encoding to 'YearMade_Bucket'
#year_bucket_encoder = OrdinalEncoder(categories=[["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]])
#df_filtered["YearMade_Bucket"] = year_bucket_encoder.fit_transform(df_filtered[["YearMade_Bucket"]])

# Identify high, moderate, and low cardinality categorical columns
high_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() >= 50]
moderate_cardinality_cols = [col for col in categorical_cols if 10 <= df_filtered[col].nunique() < 50]
low_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() < 10]

# Fill missing categorical values with "Unknown"
for col in categorical_cols:
    df_filtered[col] = df_filtered[col].fillna("Unknown")

# Encoding strategy

# Apply One-Hot Encoding (OHE) to low-cardinality categories
df_encoded = pd.get_dummies(df_filtered, columns=low_cardinality_cols, drop_first=True)

# Apply Ordinal Encoding to moderate-cardinality categories
if moderate_cardinality_cols:
    ord_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    df_encoded[moderate_cardinality_cols] = ord_encoder.fit_transform(df_encoded[moderate_cardinality_cols])

# Apply Frequency Encoding to high-cardinality categories
for col in high_cardinality_cols:
    freq_encoding = df_encoded[col].value_counts().to_dict()
    df_encoded[col] = df_encoded[col].map(freq_encoding)

# Drop rows with missing SalePrice (target variable)
df_encoded = df_encoded.dropna(subset=['SalePrice'])

# Split into features and target
X = df_encoded.drop(columns=['SalePrice'])
y = df_encoded['SalePrice']

# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

# Display model performance
mae, rmse

C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\3362339656.py:201: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path_train)
C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\3362339656.py:99: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df['Engine_Horsepower_Imputed'] = df['Engine_Horsepower'].combine_first(df['Extracted_Horsepower'])
C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\3362339656.py:66: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beha

update_YearMade START


C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\3362339656.py:238: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1980. 1980. 1994. ... 2003. 2004. 2004.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\3362339656.py:244: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Yea

update_YearMade END


(4669.066033377101, 7427.552140505497)

In [12]:
# Optional: Log feature importances
importances = rf_model.feature_importances_

# Create a zip of feature names and feature importances
feature_zip = zip(rf_model.feature_names_in_, rf_model.feature_importances_)

# Sort the zip by feature importance in descending order
sorted_feature_zip = sorted(feature_zip, key=lambda x: x[1], reverse=True)

# Print the sorted feature names, importances, and index
for idx, (feat, importance) in enumerate(sorted_feature_zip, 1):
  print({f"feature_{idx}_{feat}_importance" : importance})

{'feature_1_Enclosure_cat_importance': 0.2892227476528825}
{'feature_2_Model_Category_Mid Price Models_importance': 0.09991882770386451}
{'feature_3_Model_Category_Low Price Models_importance': 0.09986489428263046}
{'feature_4_Predicted_Model_Category_Low Price Models_importance': 0.09774432475715128}
{'feature_5_Predicted_Model_Category_Mid Price Models_importance': 0.09288049208433057}
{'feature_6_YearMade_importance': 0.061722771285000866}
{'feature_7_ModelID_importance': 0.03995959510981458}
{'feature_8_sale_year_importance': 0.03188214953969669}
{'feature_9_MachineID_importance': 0.020630266840347355}
{'feature_10_Engine_Horsepower_Imputed_importance': 0.0160786865685698}
{'feature_11_sale_month_importance': 0.016056225844659454}
{'feature_12_sale_day_importance': 0.014334765789286658}
{'feature_13_Extracted_Horsepower_importance': 0.014030403423741022}
{'feature_14_MachineHoursCurrentMeter_importance': 0.011001714799535735}
{'feature_15_ProductSize_Imputed_Large / Medium_importan

In [ ]:
def create_model_category_mapping(training_df):
    """
    Creates a mapping of ModelID to Model_Category based on fiModelDesc and SalePrice in training data.
    
    Parameters:
    training_df (pd.DataFrame): Training dataset containing fiModelDesc and SalePrice.
    
    Returns:
    dict: A dictionary mapping ModelID to Model_Category.
    """
    # Compute average SalePrice per fiModelDesc in training data
    model_avg_price = training_df.groupby("fiModelDesc")["SalePrice"].mean()
    
    # Define price categories
    def categorize_price(price):
        if price < 20000:
            return "Low Price Models"
        elif 20000 <= price < 60000:
            return "Mid Price Models"
        else:
            return "High Price Models"
    
    # Map fiModelDesc to categories
    model_category_mapping = model_avg_price.apply(categorize_price).to_dict()
    
    # Create ModelID to category mapping using fiModelDesc
    training_df["Model_Category"] = training_df["fiModelDesc"].map(model_category_mapping)
    modelid_to_category = training_df.set_index("ModelID")["Model_Category"].to_dict()
    
    return modelid_to_category, model_category_mapping

def categorize_model_id(df, modelid_to_category, model_category_mapping):
    """
    Categorizes ModelID based on precomputed price categories.
    If ModelID is not found, falls back to fiModelDesc categorization.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing ModelID and fiModelDesc columns.
    modelid_to_category (dict): Mapping of ModelID to Model_Category.
    model_category_mapping (dict): Mapping of fiModelDesc to Model_Category for fallback.
    
    Returns:
    pd.Series: A Series with categorized model price labels.
    """
    df["Predicted_Model_Category"] = df["ModelID"].map(modelid_to_category)
    
    # Handle missing ModelID by checking fiModelDesc mapping
    missing_mask = df["Predicted_Model_Category"].isna()
    df.loc[missing_mask, "Predicted_Model_Category"] = df.loc[missing_mask, "fiModelDesc"].map(model_category_mapping)
    
    # Handle any remaining missing values by assigning 'Unknown'
    df["Predicted_Model_Category"].fillna("Unknown", inplace=True)
    
    return df["Predicted_Model_Category"]

# Example usage:
# modelid_to_category, model_category_mapping = create_model_category_mapping(training_df)
# test_df["Predicted_Model_Category"] = categorize_model_id(test_df, modelid_to_category, model_category_mapping)
# Create mappings from training data
modelid_to_category, model_category_mapping = create_model_category_mapping(df)

# Apply categorization to test data
df["Predicted_Model_Category"] = categorize_model_id(df, modelid_to_category, model_category_mapping)

In [ ]:
# Define price-based categories for ProductGroup
def categorize_product_group(price):
    if price < 20000:
        return "Low Price Group"
    elif 20000 <= price < 40000:
        return "Mid Price Group"
    else:
        return "High Price Group"

# Compute the average price for each ProductGroup
product_group_avg_price = df.groupby("ProductGroup")["SalePrice"].mean()

# Map each ProductGroup to a price category
df["ProductGroup_Category"] = df["ProductGroup"].map(product_group_avg_price).apply(categorize_product_group)



In [ ]:
# Define price-based categories for Steering_Controls
def categorize_steering_controls(value):
    if value in ["Four Wheel Standard"]:
        return "Low Price Steering"
    elif value in ["Conventional"]:
        return "Mid Price Steering"
    elif value in ["Command Control"]:
        return "High Price Steering"
    else:
        return "Unknown"

# Apply categorization
df["Steering_Controls_Category"] = df["Steering_Controls"].apply(categorize_steering_controls)

# Verify the distribution of new categories
df["Steering_Controls_Category"].value_counts()


In [ ]:
# Define price-based categories for Transmission
def categorize_transmission(value):
    if value in ["Powershuttle", "Standard", "Direct Drive"]:
        return "Low Price Transmission"
    elif value in ["Autoshift", "Hydrostatic"]:
        return "Mid Price Transmission"
    elif value in ["Powershift", "None or Unspecified", "AutoShift"]:
        return "High Price Transmission"
    else:
        return "Unknown"

# Apply categorization
df["Transmission_Category"] = df["Transmission"].apply(categorize_transmission)

# Verify the distribution of new categories
df["Transmission_Category"].value_counts()


In [13]:
# Re-import necessary libraries due to execution reset
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

# Define price-based categories for Track_Type
def categorize_track_type(value):
    if value in ["Rubber"]:
        return "Low Price Track"
    elif value in ["Steel"]:
        return "High Price Track"
    else:
        return "Unknown"


# Define price-based categories for Transmission
def categorize_transmission(value):
    if value in ["Powershuttle", "Standard", "Direct Drive"]:
        return "Low Price Transmission"
    elif value in ["Autoshift", "Hydrostatic"]:
        return "Mid Price Transmission"
    elif value in ["Powershift", "None or Unspecified", "AutoShift"]:
        return "High Price Transmission"
    else:
        return "Unknown"


# Define price-based categories for Steering_Controls
def categorize_steering_controls(value):
    if value in ["Four Wheel Standard"]:
        return "Low Price Steering"
    elif value in ["Conventional"]:
        return "Mid Price Steering"
    elif value in ["Command Control"]:
        return "High Price Steering"
    else:
        return "Unknown"



# Define price-based categories for ProductGroup
def categorize_product_group(price):
    if price < 20000:
        return "Low Price Group"
    elif 20000 <= price < 40000:
        return "Mid Price Group"
    else:
        return "High Price Group"


def create_model_category_mapping(training_df):
    """
    Creates a mapping of ModelID to Model_Category based on fiModelDesc and SalePrice in training data.
    
    Parameters:
    training_df (pd.DataFrame): Training dataset containing fiModelDesc and SalePrice.
    
    Returns:
    dict: A dictionary mapping ModelID to Model_Category.
    """
    # Compute average SalePrice per fiModelDesc in training data
    model_avg_price = training_df.groupby("fiModelDesc")["SalePrice"].mean()
    
    # Define price categories
    def categorize_price(price):
        if price < 20000:
            return "Low Price Models"
        elif 20000 <= price < 60000:
            return "Mid Price Models"
        else:
            return "High Price Models"
    
    # Map fiModelDesc to categories
    model_category_mapping = model_avg_price.apply(categorize_price).to_dict()
    
    # Create ModelID to category mapping using fiModelDesc
    training_df["Model_Category"] = training_df["fiModelDesc"].map(model_category_mapping)
    modelid_to_category = training_df.set_index("ModelID")["Model_Category"].to_dict()
    
    return modelid_to_category, model_category_mapping

def categorize_model_id(df, modelid_to_category, model_category_mapping):
    """
    Categorizes ModelID based on precomputed price categories.
    If ModelID is not found, falls back to fiModelDesc categorization.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing ModelID and fiModelDesc columns.
    modelid_to_category (dict): Mapping of ModelID to Model_Category.
    model_category_mapping (dict): Mapping of fiModelDesc to Model_Category for fallback.
    
    Returns:
    pd.Series: A Series with categorized model price labels.
    """
    df["Predicted_Model_Category"] = df["ModelID"].map(modelid_to_category)
    
    # Handle missing ModelID by checking fiModelDesc mapping
    missing_mask = df["Predicted_Model_Category"].isna()
    df.loc[missing_mask, "Predicted_Model_Category"] = df.loc[missing_mask, "fiModelDesc"].map(model_category_mapping)
    
    # Handle any remaining missing values by assigning 'Unknown'
    df["Predicted_Model_Category"].fillna("Unknown", inplace=True)
    
    return df["Predicted_Model_Category"]



def extract_horsepower(df):
    """
    Extracts and imputes missing Horsepower values from fiProductClassDesc.
    Optimized for performance using vectorized operations.
    """
    def extract_numeric_range(value):
        """Extracts the average horsepower value from a range like '100 to 120 Horsepower'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:
            return float(numbers[0])
        return np.nan
    
    # Identify rows with Horsepower in fiProductClassDesc
    mask_horsepower = df['fiProductClassDesc'].str.contains("Horsepower", na=False)
    
    # Extract horsepower values
    df.loc[mask_horsepower, 'Horsepower_Unit_Type'] = 'Horsepower'
    df.loc[mask_horsepower, 'Extracted_Horsepower'] = df.loc[mask_horsepower, 'fiProductClassDesc'].apply(extract_numeric_range)
    
    # Handle 'Variable' and 'No' values in Engine_Horsepower
    df.loc[df['Engine_Horsepower'].isin(['Variable', 'No']), 'Engine_Horsepower'] = np.nan
    
    # Impute missing Engine_Horsepower values where Extracted_Horsepower is available
    df['Engine_Horsepower_Imputed'] = df['Engine_Horsepower'].combine_first(df['Extracted_Horsepower'])
    
    # Fill remaining NaN values with median horsepower
    #df['Engine_Horsepower_Imputed'].fillna(df['Engine_Horsepower_Imputed'].median(), inplace=True)
    df.drop(columns='Engine_Horsepower',inplace=True)
    
    return df

def preprocess_product_size(df):
    """
    Function to preprocess and impute missing ProductSize values based on fiProductClassDesc.
    
    Steps:
    1. Extract Product Type and Metric Tons / Horsepower from fiProductClassDesc.
    2. Identify Unit Type (Metric Tons, Horsepower, or Lb Operating Capacity).
    3. Convert Metric Tons / Horsepower to numerical values.
    4. Use Metric Tons to impute missing ProductSize values.
    """
    
    # Step 1: Extract Product Type and Size Descriptor
    def split_product_desc(desc):
        if pd.isna(desc):
            return pd.NA, pd.NA
        match = re.search(r'(.+?)-\s*([\d\.]+.*)', desc)
        if match:
            return match.group(1).strip(), match.group(2).strip()
        return desc.strip(), pd.NA  # If no match, return full description as product
    
    df[['Product_Type', 'Metric_Tons_HP']] = df['fiProductClassDesc'].apply(lambda x: pd.Series(split_product_desc(x)))
    
    # Step 2: Identify Unit Type
    def extract_unit_type(desc):
        if pd.isna(desc):
            return pd.NA
        if "Metric Tons" in desc:
            return "Metric Tons"
        elif "Horsepower" in desc:
            return "Horsepower"
        elif "Lb Operating Capacity" in desc:
            return "Lb Operating Capacity"
        return pd.NA
    
    df["Unit_Type"] = df["Metric_Tons_HP"].apply(extract_unit_type)
    
    # Step 3: Convert Metric Tons to numerical values
    def extract_numeric_range(value):
        """Extracts the average value from a range like '12.0 to 14.0 Metric Tons'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:  # If range exists, take the average
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:  # If only one number exists, use it
            return float(numbers[0])
        return np.nan
    
    df['Metric_Tons_Value'] = df['Metric_Tons_HP'].apply(lambda x: extract_numeric_range(x) if isinstance(x, str) else np.nan)
    
    # Step 4: Impute missing ProductSize values using Metric Tons
    def impute_product_size(row):
        if pd.isna(row['ProductSize']) and not pd.isna(row['Metric_Tons_Value']) and (row['Metric_Tons_Value']=='Metric Tons'):
            if row['Metric_Tons_Value']  <= 5:
                return 'Mini'
            elif 5 < row['Metric_Tons_Value'] <= 75:
                return 'Compact'
            elif 20 < row['Metric_Tons_Value'] <= 50:
                return 'Large / Medium'
            elif 75 < row['Metric_Tons_Value'] <= 200:
                return 'Medium'
            elif row['Metric_Tons_Value'] > 200:
                return 'Large'
        return row['ProductSize']
    
    df['ProductSize_Imputed'] = df.apply(impute_product_size, axis=1)
    df.drop(columns=['ProductSize','Metric_Tons_Value','Metric_Tons_HP','Unit_Type'],inplace=True)

    
    
    return df
def Enclosure_fun(df):
    list1=['EROPS','OROPS','EROPS AC']
    list2=['NO ROPS','None or Unspecified']
    df['Enclosure_cat']=0
    df.loc[df['Enclosure'].isin(list1), 'Enclosure_cat'] = 1
    df.loc[df['Enclosure'].isin(list2), 'Enclosure_cat'] = 0
    df.loc[df['Enclosure']=='EROPS w AC','Enclosure_cat']=2
    df.drop(columns='Enclosure',inplace=True)
    return df
# Define categories based on observed price trends
def categorize_hydraulics(value):
    if value in ["Missing", "Auxiliary", "Standard"]:
        return "Basic"
    elif value in ["2 Valve", "None or Unspecified", "3 Valve"]:
        return "Mid-Level"
    else:
        return "Advanced"



# Reload the dataset
ProductSize_path=r'C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/ProductSize_Null_fix_model.pkl'
file_path_train = r"C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/Train.csv"
df = pd.read_csv(file_path_train)
#df=df[['SalesID','saledate',"Enclosure","YearMade","ModelID","MachineID","Ripper","ProductGroup","ProductSize","ProductGroupDesc","Tire_Size","MachineHoursCurrentMeter","Blade_Type","ProductGroupDesc","Travel_Controls","auctioneerID","datasource","Drive_System","Pushblock","Tire_Size","Ride_Control","UsageBand","Coupler","Engine_Horsepower_Imputed"]].copy()

# Convert 'saledate' to datetime and extract year, month, and day
df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')
df['sale_year'] = df['saledate'].dt.year
df['sale_month'] = df['saledate'].dt.month
df['sale_day'] = df['saledate'].dt.day

# Apply categorization
df["Hydraulics_Category"] = df["Hydraulics"].apply(categorize_hydraulics)
# Apply categorization
df["Steering_Controls_Category"] = df["Steering_Controls"].apply(categorize_steering_controls)
# Apply categorization
df["Transmission_Category"] = df["Transmission"].apply(categorize_transmission)

df=Enclosure_fun(df)
df=extract_horsepower(df)

# Create mappings from training data only in saleprice 
modelid_to_category, model_category_mapping = create_model_category_mapping(df) # need to upload the modelid_to_category, model_category_mapping in the test

# Apply categorization to test data
df["Predicted_Model_Category"] = categorize_model_id(df, modelid_to_category, model_category_mapping) # need to upload the modelid_to_category, model_category_mapping in the test

# Compute the average price for each ProductGroup
# Apply categorization
df["Track_Type_Category"] = df["Track_Type"].apply(categorize_track_type)

product_group_avg_price = df.groupby("ProductGroup")["SalePrice"].mean()

# Map each ProductGroup to a price category
df["ProductGroup_Category"] = df["ProductGroup"].map(product_group_avg_price).apply(categorize_product_group)# need to upload the categorize_product_group in the test
df=preprocess_product_size(df)

# Define function to update 'YearMade' and create 'YearMade_Bucket'
def update_YearMade(df):
    print("update_YearMade START")
    
    # Compute median YearMade for each ModelID
    model_medians = df.loc[df['YearMade'] > 1000].groupby('ModelID')['YearMade'].median()
    
    # Update YearMade where it is 1000 using ModelID median
    df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
    
    # Compute overall median YearMade excluding 1000 values
    overall_median = df.loc[df['YearMade'] > 1000, 'YearMade'].median()
    
    # Replace any remaining 1000 values with overall median
    df['YearMade'].fillna(overall_median, inplace=True)
        
    # Bucketize YearMade
    bins = [0, 1980, 1995, 2005, 2010, 2025]
    labels = ["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]
    df['YearMade_Bucket'] = pd.cut(df['YearMade'], bins=bins, labels=labels, right=True)
    
    print("update_YearMade END")
    return df

# Apply YearMade update
df = update_YearMade(df)

#df.drop(columns=['fiProductClassDesc','Hydraulics','fiSecondaryDesc','auctioneerID','fiModelDesc','fiBaseModel','ProductGroupDesc','MachineID','ProductGroup'],inplace=True)


# Filter dataset to only include the last 5 years
recent_years = df['sale_year'].dropna().unique()
recent_years.sort()
selected_years = recent_years[-5:]
df_filtered = df[df['sale_year'].isin(selected_years)]

# Drop irrelevant columns: 'SalesID', 'saledate'
high_cardinality_cols = [col for col in df_filtered.select_dtypes(include=['object']).columns if df_filtered[col].nunique() > 50]
df_filtered = df_filtered.drop(columns=['SalesID', 'saledate'] + high_cardinality_cols, errors='ignore')

# Identify categorical columns
categorical_cols = df_filtered.select_dtypes(include=['object']).columns

df_filtered["YearMade_Bucket"] = df_filtered["YearMade_Bucket"].astype(str).fillna("Unknown")

# Define category order and apply Ordinal Encoding
year_bucket_encoder = OrdinalEncoder(
    categories=[["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025", "Unknown"]],
    handle_unknown="use_encoded_value",
    unknown_value=-1
)

df_filtered["YearMade_Bucket"] = year_bucket_encoder.fit_transform(df_filtered[["YearMade_Bucket"]])
# Apply Ordinal Encoding to 'YearMade_Bucket'
#year_bucket_encoder = OrdinalEncoder(categories=[["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]])
#df_filtered["YearMade_Bucket"] = year_bucket_encoder.fit_transform(df_filtered[["YearMade_Bucket"]])

# Identify high, moderate, and low cardinality categorical columns
high_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() >= 50]
moderate_cardinality_cols = [col for col in categorical_cols if 10 <= df_filtered[col].nunique() < 50]
low_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() < 10]

# Fill missing categorical values with "Unknown"
for col in categorical_cols:
    df_filtered[col] = df_filtered[col].fillna("Unknown")

# Encoding strategy

# Apply One-Hot Encoding (OHE) to low-cardinality categories
df_encoded = pd.get_dummies(df_filtered, columns=low_cardinality_cols, drop_first=True)

# Apply Ordinal Encoding to moderate-cardinality categories
if moderate_cardinality_cols:
    ord_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
    df_encoded[moderate_cardinality_cols] = ord_encoder.fit_transform(df_encoded[moderate_cardinality_cols])

# Apply Frequency Encoding to high-cardinality categories
for col in high_cardinality_cols:
    freq_encoding = df_encoded[col].value_counts().to_dict()
    df_encoded[col] = df_encoded[col].map(freq_encoding)

# Drop rows with missing SalePrice (target variable)
df_encoded = df_encoded.dropna(subset=['SalePrice'])

# Split into features and target
X = df_encoded.drop(columns=['SalePrice'])
y = df_encoded['SalePrice']

# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

# Display model performance
mae, rmse

C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\35781373.py:236: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path_train)
C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\35781373.py:134: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df['Engine_Horsepower_Imputed'] = df['Engine_Horsepower'].combine_first(df['Extracted_Horsepower'])
C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\35781373.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves 

update_YearMade START


C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\35781373.py:279: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1980. 1980. 1994. ... 2003. 2004. 2004.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
C:\Users\eitanb\AppData\Local\Temp\ipykernel_12692\35781373.py:285: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['YearMad

update_YearMade END


(4666.387783170206, 7416.27935846593)

In [ ]:
# Define final categorization for Steering_Controls
def categorize_steering_controls_final(value):
    if value in ["Four Wheel Standard", "Conventional"]:
        return "Mid Price Steering"
    elif value in ["Command Control"]:
        return "High Price Steering"
    else:
        return "Unknown"

# Apply categorization
df["Steering_Controls_Category"] = df["Steering_Controls"].apply(categorize_steering_controls_final)

# Verify the distribution of the new categories
df["Steering_Controls_Category"].value_counts()


In [35]:
# Re-import necessary libraries due to execution reset
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

# Define final categorization for Steering_Controls
def categorize_steering_controls_final(value):
    if value in ["Four Wheel Standard", "Conventional"]:
        return "Mid Price Steering"
    elif value in ["Command Control"]:
        return "High Price Steering"
    else:
        return "Unknown"



# Define price-based categories for Track_Type
def categorize_track_type(value):
    if value in ["Rubber"]:
        return "Low Price Track"
    elif value in ["Steel"]:
        return "High Price Track"
    else:
        return "Unknown"


# Define price-based categories for Transmission
def categorize_transmission(value):
    if value in ["Powershuttle", "Standard", "Direct Drive"]:
        return "Low Price Transmission"
    elif value in ["Autoshift", "Hydrostatic"]:
        return "Mid Price Transmission"
    elif value in ["Powershift", "None or Unspecified", "AutoShift"]:
        return "High Price Transmission"
    else:
        return "Unknown"


# Define price-based categories for Steering_Controls
def categorize_steering_controls(value):
    if value in ["Four Wheel Standard"]:
        return "Low Price Steering"
    elif value in ["Conventional"]:
        return "Mid Price Steering"
    elif value in ["Command Control"]:
        return "High Price Steering"
    else:
        return "Unknown"



# Define price-based categories for ProductGroup
def categorize_product_group(price):
    if price < 20000:
        return "Low Price Group"
    elif 20000 <= price < 40000:
        return "Mid Price Group"
    else:
        return "High Price Group"


def create_model_category_mapping(df,model_avg_price):
    """
    Creates a mapping of ModelID to Model_Category based on fiModelDesc and SalePrice in training data.
    
    Parameters:
    training_df (pd.DataFrame): Training dataset containing fiModelDesc and SalePrice.
    
    Returns:
    dict: A dictionary mapping ModelID to Model_Category.
    """
    # Compute average SalePrice per fiModelDesc in training data
    
    
    # Define price categories
    def categorize_price(price):
        if price < 20000:
            return "Low Price Models"
        elif 20000 <= price < 60000:
            return "Mid Price Models"
        else:
            return "High Price Models"
    
    # Map fiModelDesc to categories
    model_category_mapping = model_avg_price.apply(categorize_price).to_dict()
    
    # Create ModelID to category mapping using fiModelDesc
    df["Model_Category"] = df["fiModelDesc"].map(model_category_mapping)
    modelid_to_category = df.set_index("ModelID")["Model_Category"].to_dict()
    
    return modelid_to_category, model_category_mapping

def categorize_model_id(df, modelid_to_category, model_category_mapping):
    """
    Categorizes ModelID based on precomputed price categories.
    If ModelID is not found, falls back to fiModelDesc categorization.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing ModelID and fiModelDesc columns.
    modelid_to_category (dict): Mapping of ModelID to Model_Category.
    model_category_mapping (dict): Mapping of fiModelDesc to Model_Category for fallback.
    
    Returns:
    pd.Series: A Series with categorized model price labels.
    """
    df["Predicted_Model_Category"] = df["ModelID"].map(modelid_to_category)
    
    # Handle missing ModelID by checking fiModelDesc mapping
    missing_mask = df["Predicted_Model_Category"].isna()
    df.loc[missing_mask, "Predicted_Model_Category"] = df.loc[missing_mask, "fiModelDesc"].map(model_category_mapping)
    
    # Handle any remaining missing values by assigning 'Unknown'
    df["Predicted_Model_Category"].fillna("Unknown", inplace=True)
    
    return df["Predicted_Model_Category"]



def extract_horsepower(df):
    """
    Extracts and imputes missing Horsepower values from fiProductClassDesc.
    Optimized for performance using vectorized operations.
    """
    def extract_numeric_range(value):
        """Extracts the average horsepower value from a range like '100 to 120 Horsepower'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:
            return float(numbers[0])
        return np.nan
    
    # Identify rows with Horsepower in fiProductClassDesc
    mask_horsepower = df['fiProductClassDesc'].str.contains("Horsepower", na=False)
    
    # Extract horsepower values
    df.loc[mask_horsepower, 'Horsepower_Unit_Type'] = 'Horsepower'
    df.loc[mask_horsepower, 'Extracted_Horsepower'] = df.loc[mask_horsepower, 'fiProductClassDesc'].apply(extract_numeric_range)
    
    # Handle 'Variable' and 'No' values in Engine_Horsepower
    df.loc[df['Engine_Horsepower'].isin(['Variable', 'No']), 'Engine_Horsepower'] = np.nan
    
    # Impute missing Engine_Horsepower values where Extracted_Horsepower is available
    df['Engine_Horsepower_Imputed'] = df['Engine_Horsepower'].combine_first(df['Extracted_Horsepower'])
    
    # Fill remaining NaN values with median horsepower
    #df['Engine_Horsepower_Imputed'].fillna(df['Engine_Horsepower_Imputed'].median(), inplace=True)
    df.drop(columns='Engine_Horsepower',inplace=True)
    
    return df

def preprocess_product_size(df):
    """
    Function to preprocess and impute missing ProductSize values based on fiProductClassDesc.
    
    Steps:
    1. Extract Product Type and Metric Tons / Horsepower from fiProductClassDesc.
    2. Identify Unit Type (Metric Tons, Horsepower, or Lb Operating Capacity).
    3. Convert Metric Tons / Horsepower to numerical values.
    4. Use Metric Tons to impute missing ProductSize values.
    """
    
    # Step 1: Extract Product Type and Size Descriptor
    def split_product_desc(desc):
        if pd.isna(desc):
            return pd.NA, pd.NA
        match = re.search(r'(.+?)-\s*([\d\.]+.*)', desc)
        if match:
            return match.group(1).strip(), match.group(2).strip()
        return desc.strip(), pd.NA  # If no match, return full description as product
    
    df[['Product_Type', 'Metric_Tons_HP']] = df['fiProductClassDesc'].apply(lambda x: pd.Series(split_product_desc(x)))
    
    # Step 2: Identify Unit Type
    def extract_unit_type(desc):
        if pd.isna(desc):
            return pd.NA
        if "Metric Tons" in desc:
            return "Metric Tons"
        elif "Horsepower" in desc:
            return "Horsepower"
        elif "Lb Operating Capacity" in desc:
            return "Lb Operating Capacity"
        return pd.NA
    
    df["Unit_Type"] = df["Metric_Tons_HP"].apply(extract_unit_type)
    
    # Step 3: Convert Metric Tons to numerical values
    def extract_numeric_range(value):
        """Extracts the average value from a range like '12.0 to 14.0 Metric Tons'."""
        if pd.isna(value) or not isinstance(value, str):
            return np.nan
        numbers = re.findall(r'[\d\.]+', value)
        if len(numbers) == 2:  # If range exists, take the average
            return (float(numbers[0]) + float(numbers[1])) / 2
        elif len(numbers) == 1:  # If only one number exists, use it
            return float(numbers[0])
        return np.nan
    
    df['Metric_Tons_Value'] = df['Metric_Tons_HP'].apply(lambda x: extract_numeric_range(x) if isinstance(x, str) else np.nan)
    
    # Step 4: Impute missing ProductSize values using Metric Tons
    def impute_product_size(row):
        if pd.isna(row['ProductSize']) and not pd.isna(row['Metric_Tons_Value']) and (row['Metric_Tons_Value']=='Metric Tons'):
            if row['Metric_Tons_Value']  <= 5:
                return 'Mini'
            elif 5 < row['Metric_Tons_Value'] <= 75:
                return 'Compact'
            elif 20 < row['Metric_Tons_Value'] <= 50:
                return 'Large / Medium'
            elif 75 < row['Metric_Tons_Value'] <= 200:
                return 'Medium'
            elif row['Metric_Tons_Value'] > 200:
                return 'Large'
        return row['ProductSize']
    
    df['ProductSize_Imputed'] = df.apply(impute_product_size, axis=1)
    df.drop(columns=['ProductSize','Metric_Tons_Value','Metric_Tons_HP','Unit_Type'],inplace=True)

    
    
    return df
def Enclosure_fun(df):
    list1=['EROPS','OROPS','EROPS AC']
    list2=['NO ROPS','None or Unspecified']
    df['Enclosure_cat']=0
    df.loc[df['Enclosure'].isin(list1), 'Enclosure_cat'] = 1
    df.loc[df['Enclosure'].isin(list2), 'Enclosure_cat'] = 0
    df.loc[df['Enclosure']=='EROPS w AC','Enclosure_cat']=2
    df.drop(columns='Enclosure',inplace=True)
    return df
# Define categories based on observed price trends
def categorize_hydraulics(value):
    if value in ["Missing", "Auxiliary", "Standard"]:
        return "Basic"
    elif value in ["2 Valve", "None or Unspecified", "3 Valve"]:
        return "Mid-Level"
    else:
        return "Advanced"



# Reload the dataset
    

#df=df[['SalesID','saledate',"Enclosure","YearMade","ModelID","MachineID","Ripper","ProductGroup","ProductSize","ProductGroupDesc","Tire_Size","MachineHoursCurrentMeter","Blade_Type","ProductGroupDesc","Travel_Controls","auctioneerID","datasource","Drive_System","Pushblock","Tire_Size","Ride_Control","UsageBand","Coupler","Engine_Horsepower_Imputed"]].copy()
def saledate(df):
    # Convert 'saledate' to datetime and extract year, month, and day
    df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')
    df['sale_year'] = df['saledate'].dt.year
    df['sale_month'] = df['saledate'].dt.month
    df['sale_day'] = df['saledate'].dt.day
    return df



def preprocess_data(df,modelid_to_category, model_category_mapping,product_group_avg_price):
    # Apply categorization
    df["Hydraulics_Category"] = df["Hydraulics"].apply(categorize_hydraulics)
    # Apply categorization
    df["Steering_Controls_Category"] = df["Steering_Controls"].apply(categorize_steering_controls)
    # Apply categorization
    df["Transmission_Category"] = df["Transmission"].apply(categorize_transmission)
    
    # Apply categorization
    df["Steering_Controls_Category"] = df["Steering_Controls"].apply(categorize_steering_controls_final)
    # Apply categorization to test data
    # Apply categorization to test dat
    df["Predicted_Model_Category"] = categorize_model_id(df, modelid_to_category, model_category_mapping) # need to upload the modelid_to_category, model_category_mapping in the test
    df["Track_Type_Category"] = df["Track_Type"].apply(categorize_track_type)
    # Map each ProductGroup to a price category
    df["ProductGroup_Category"] = df["ProductGroup"].map(product_group_avg_price).apply(categorize_product_group) # need to upload the categorize_product_group in the test



    return df

# Define function to update 'YearMade' and create 'YearMade_Bucket'
def update_YearMade(df):
    print("update_YearMade START")
    
    # Compute median YearMade for each ModelID
    model_medians = df.loc[df['YearMade'] > 1000].groupby('ModelID')['YearMade'].median()
    
    # Update YearMade where it is 1000 using ModelID median
    df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
    
    # Compute overall median YearMade excluding 1000 values
    overall_median = df.loc[df['YearMade'] > 1000, 'YearMade'].median()
    
    # Replace any remaining 1000 values with overall median
    df['YearMade'].fillna(overall_median, inplace=True)
        
    # Bucketize YearMade
    bins = [0, 1980, 1995, 2005, 2010, 2025]
    labels = ["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025"]
    df['YearMade_Bucket'] = pd.cut(df['YearMade'], bins=bins, labels=labels, right=True)

    df_filtered["YearMade_Bucket"] = df["YearMade_Bucket"].astype(str).fillna("Unknown")
    
    # Define category order and apply Ordinal Encoding
    year_bucket_encoder = OrdinalEncoder(
        categories=[["Before 1980", "1980-1995", "1996-2005", "2006-2010", "2011-2025", "Unknown"]],
        handle_unknown="use_encoded_value",
        unknown_value=-1
    )
    
    df["YearMade_Bucket"] = year_bucket_encoder.fit_transform(df[["YearMade_Bucket"]])
    
    print("update_YearMade END")
    return df
def pre_train(df):
    model_avg_price = df.groupby("fiModelDesc")["SalePrice"].mean()
    product_group_avg_price = df.groupby("ProductGroup")["SalePrice"].mean()

    return model_avg_price,product_group_avg_price
def Pre_train_test_model(df):
    # Filter dataset to only include the last 5 years
    recent_years = df['sale_year'].dropna().unique()
    recent_years.sort()
    #selected_years = recent_years[-5:]
    #df_filtered = df[df['sale_year'].isin(selected_years)]
    df_filtered=df.copy()
    
    # Drop irrelevant columns: 'SalesID', 'saledate'
    high_cardinality_cols = [col for col in df_filtered.select_dtypes(include=['object']).columns if df_filtered[col].nunique() > 50]
    df_filtered = df_filtered.drop(columns=['SalesID', 'saledate'] + high_cardinality_cols, errors='ignore')
    
    # Identify categorical columns
    categorical_cols = df_filtered.select_dtypes(include=['object']).columns
    
    
    
    # Identify high, moderate, and low cardinality categorical columns
    high_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() >= 50]
    moderate_cardinality_cols = [col for col in categorical_cols if 10 <= df_filtered[col].nunique() < 50]
    low_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() < 10]
    
    # Fill missing categorical values with "Unknown"
    for col in categorical_cols:
        df_filtered[col] = df_filtered[col].fillna("Unknown")
    
    # Encoding strategy
    
    # Apply One-Hot Encoding (OHE) to low-cardinality categories
    df_encoded = pd.get_dummies(df_filtered, columns=low_cardinality_cols, drop_first=True)
    
    # Apply Ordinal Encoding to moderate-cardinality categories
    if moderate_cardinality_cols:
        ord_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
        df_encoded[moderate_cardinality_cols] = ord_encoder.fit_transform(df_encoded[moderate_cardinality_cols])
    
    # Apply Frequency Encoding to high-cardinality categories
    for col in high_cardinality_cols:
        freq_encoding = df_encoded[col].value_counts().to_dict()
        df_encoded[col] = df_encoded[col].map(freq_encoding)
    
    # Drop rows with missing SalePrice (target variable)
    df_encoded = df_encoded.dropna(subset=['SalePrice'])

    return df_encoded
def train_run(df_encoded):

    # Split into features and target
    X = df_encoded.drop(columns=['SalePrice'])
    y = df_encoded['SalePrice']
    
    # Split data into train and test sets
    from sklearn.model_selection import train_test_split
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train a Random Forest model
    from sklearn.ensemble import RandomForestRegressor
    
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    
    # Evaluate the model
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    
    y_pred = rf_model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    
    # Display model performance
    print(mae, rmse)
    return rf_model


#-------------------------------------------------------------------------------------------------


In [39]:
#def main()
ProductSize_path=r'C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/ProductSize_Null_fix_model.pkl'
file_path_train = r"C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/Train.csv"
df = pd.read_csv(file_path_train)[['SalesID', 'SalePrice',  'ModelID',
        'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 
       'Drive_System', 'Enclosure', 
        'Transmission', 'Turbocharged',  'Engine_Horsepower', 'Hydraulics',
         'Tire_Size',
       'Track_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls','fiModelDesc']]

# Apply YearMade update
df = update_YearMade(df)
df= saledate(df)

model_avg_price,product_group_avg_price=pre_train(df)
# Create mappings from training data only in saleprice 
modelid_to_category, model_category_mapping = create_model_category_mapping(df,model_avg_price) # need to upload the modelid_to_category, model_category_mapping in the test


df=preprocess_data(df,modelid_to_category, model_category_mapping,product_group_avg_price)
df=Enclosure_fun(df)
df=extract_horsepower(df)

# Map each ProductGroup to a price category
df=preprocess_product_size(df)






# Compute the average price for each ProductGroup
# Apply categorization


df=Pre_train_test_model(df)


#df.drop(columns=['fiProductClassDesc','Hydraulics','fiSecondaryDesc','auctioneerID','fiModelDesc','fiBaseModel','ProductGroupDesc','MachineID','ProductGroup'],inplace=True)



rf_model=train_run(df_encoded)

C:\Users\eitanb\AppData\Local\Temp\ipykernel_22792\3751923447.py:4: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path_train)[['SalesID', 'SalePrice',  'ModelID',


update_YearMade START


C:\Users\eitanb\AppData\Local\Temp\ipykernel_22792\908198221.py:287: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1980. 1980. 1994. ... 2003. 2004. 2004.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
C:\Users\eitanb\AppData\Local\Temp\ipykernel_22792\908198221.py:293: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['YearM

update_YearMade END


C:\Users\eitanb\AppData\Local\Temp\ipykernel_22792\908198221.py:112: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Predicted_Model_Category"].fillna("Unknown", inplace=True)
C:\Users\eitanb\AppData\Local\Temp\ipykernel_22792\908198221.py:145: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df['Engine_Horsepow

4540.807883070412 7310.768671236769


In [38]:
df.columns

Index(['SalesID', 'SalePrice', 'ModelID', 'YearMade',
       'MachineHoursCurrentMeter', 'UsageBand', 'saledate',
       'fiProductClassDesc', 'state', 'ProductGroup', 'Drive_System',
       'Transmission', 'Turbocharged', 'Hydraulics', 'Tire_Size', 'Track_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls',
       'fiModelDesc', 'YearMade_Bucket', 'Model_Category',
       'Hydraulics_Category', 'Steering_Controls_Category',
       'Transmission_Category', 'Predicted_Model_Category',
       'Track_Type_Category', 'ProductGroup_Category', 'Enclosure_cat',
       'Horsepower_Unit_Type', 'Extracted_Horsepower',
       'Engine_Horsepower_Imputed', 'Product_Type', 'ProductSize_Imputed'],
      dtype='object')

In [ ]:
Valid_df = pd.read_csv(file_path_train)[['SalesID',  'ModelID',
        'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 
       'Drive_System', 'Enclosure', 
        'Transmission', 'Turbocharged',  'Engine_Horsepower', 'Hydraulics',
         'Tire_Size',
       'Track_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls','fiModelDesc']]

# Apply YearMade update
df = update_YearMade(df)
df= saledate(df)

model_avg_price,product_group_avg_price=pre_train(df)
# Create mappings from training data only in saleprice 
modelid_to_category, model_category_mapping = create_model_category_mapping(df,model_avg_price) # need to upload the modelid_to_category, model_category_mapping in the test


df=preprocess_data(df,modelid_to_category, model_category_mapping,product_group_avg_price)
df=Enclosure_fun(df)
df=extract_horsepower(df)

# Map each ProductGroup to a price category
df=preprocess_product_size(df)






# Compute the average price for each ProductGroup
# Apply categorization


df=Pre_train_test_model(df)




# Ensure predictions align with the original index
predictions = rf_model.predict(df)

# Verify matching row count
if len(predictions) != len(df):
    raise ValueError("Mismatch in row count between processed data and predictions!")

# Reconstruct results DataFrame, ensuring index alignment
results = pd.DataFrame({
    "SalesID": sales_id_col.iloc[:, 0],  # Ensure correct alignment
    "SalePrice": »  # Reverse log transformation
}, index=original_index)

In [21]:
# Optional: Log feature importances
importances = rf_model.feature_importances_

# Create a zip of feature names and feature importances
feature_zip = zip(rf_model.feature_names_in_, rf_model.feature_importances_)

# Sort the zip by feature importance in descending order
sorted_feature_zip = sorted(feature_zip, key=lambda x: x[1], reverse=True)

# Print the sorted feature names, importances, and index
for idx, (feat, importance) in enumerate(sorted_feature_zip, 1):
  print({f"feature_{idx}_{feat}_importance" : importance})

{'feature_1_Predicted_Model_Category_Low Price Models_importance': 0.1859553473741895}
{'feature_2_Predicted_Model_Category_Mid Price Models_importance': 0.14865644456054966}
{'feature_3_Model_Category_Low Price Models_importance': 0.1345675563718487}
{'feature_4_Model_Category_Mid Price Models_importance': 0.10344216393111744}
{'feature_5_YearMade_importance': 0.07844089188209398}
{'feature_6_ModelID_importance': 0.0734173951672887}
{'feature_7_sale_year_importance': 0.07303127477363637}
{'feature_8_Product_Type_importance': 0.02841903265202036}
{'feature_9_sale_day_importance': 0.026005490469335507}
{'feature_10_sale_month_importance': 0.022311927108702393}
{'feature_11_Enclosure_cat_importance': 0.018173305572651913}
{'feature_12_Engine_Horsepower_Imputed_importance': 0.018094654396660054}
{'feature_13_Extracted_Horsepower_importance': 0.0180840214260408}
{'feature_14_ProductSize_Imputed_Large / Medium_importance': 0.010947471226807249}
{'feature_15_MachineHoursCurrentMeter_importan

In [29]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
file_path_train = r"C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/Train.csv"

def categorize_steering_controls(value):
    if value in ["Four Wheel Standard"]:
        return "Low Price Steering"
    elif value in ["Conventional"]:
        return "Mid Price Steering"
    elif value in ["Command Control"]:
        return "High Price Steering"
    else:
        return "Unknown"

def categorize_transmission(value):
    if value in ["Powershuttle", "Standard", "Direct Drive"]:
        return "Low Price Transmission"
    elif value in ["Autoshift", "Hydrostatic"]:
        return "Mid Price Transmission"
    elif value in ["Powershift", "None or Unspecified", "AutoShift"]:
        return "High Price Transmission"
    else:
        return "Unknown"

def categorize_track_type(value):
    if value in ["Rubber"]:
        return "Low Price Track"
    elif value in ["Steel"]:
        return "High Price Track"
    else:
        return "Unknown"

def categorize_product_group(price):
    if price < 20000:
        return "Low Price Group"
    elif 20000 <= price < 40000:
        return "Mid Price Group"
    else:
        return "High Price Group"

def categorize_hydraulics(value):
    if value in ["Missing", "Auxiliary", "Standard"]:
        return "Basic"
    elif value in ["2 Valve", "None or Unspecified", "3 Valve"]:
        return "Mid-Level"
    else:
        return "Advanced"

def create_model_category_mapping(training_df,model_avg_price):
    
    
    
    def categorize_price(price):
        if price < 20000:
            return "Low Price Models"
        elif 20000 <= price < 60000:
            return "Mid Price Models"
        else:
            return "High Price Models"
    
    model_category_mapping = model_avg_price.apply(categorize_price).to_dict()
    training_df["Model_Category"] = training_df["fiModelDesc"].map(model_category_mapping)
    modelid_to_category = training_df.set_index("ModelID")["Model_Category"].to_dict()
    
    return modelid_to_category, model_category_mapping

def categorize_model_id(df, modelid_to_category, model_category_mapping):
    df["Predicted_Model_Category"] = df["ModelID"].map(modelid_to_category)
    missing_mask = df["Predicted_Model_Category"].isna()
    df.loc[missing_mask, "Predicted_Model_Category"] = df.loc[missing_mask, "fiModelDesc"].map(model_category_mapping)
    df["Predicted_Model_Category"].fillna("Unknown", inplace=True)
    return df

def preprocess_data(df,product_group_avg_price):
    df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce')
    df['sale_year'] = df['saledate'].dt.year
    df['sale_month'] = df['saledate'].dt.month
    df['sale_day'] = df['saledate'].dt.day
    
    df["Hydraulics_Category"] = df["Hydraulics"].apply(categorize_hydraulics)
    df["Steering_Controls_Category"] = df["Steering_Controls"].apply(categorize_steering_controls)
    df["Transmission_Category"] = df["Transmission"].apply(categorize_transmission)
    df["Track_Type_Category"] = df["Track_Type"].apply(categorize_track_type)
    
    
    df["ProductGroup_Category"] = df["ProductGroup"].map(product_group_avg_price).apply(categorize_product_group)
    
    return df

def pre_train_only(file_path_train):
    df = pd.read_csv(file_path_train)[['SalesID', 'SalePrice',  'ModelID',
        'YearMade', 'MachineHoursCurrentMeter', 'UsageBand', 'saledate', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 
       'Drive_System', 'Enclosure', 
        'Transmission', 'Turbocharged',  'Engine_Horsepower', 'Hydraulics',
         'Tire_Size',
       'Track_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls','fiModelDesc']]
    product_group_avg_price = df.groupby("ProductGroup")["SalePrice"].mean()
    model_avg_price = df.groupby("fiModelDesc")["SalePrice"].mean()
    df = preprocess_data(df,product_group_avg_price)
    modelid_to_category, model_category_mapping = create_model_category_mapping(df,model_avg_price)

    df = categorize_model_id(df, modelid_to_category, model_category_mapping)
    return df,product_group_avg_price,model_avg_price,modelid_to_category, model_category_mapping

def test_only(df,product_group_avg_price,model_avg_price,modelid_to_category, model_category_mapping):
    df = preprocess_data(df,product_group_avg_price)
    modelid_to_category, model_category_mapping = create_model_category_mapping(df,model_avg_price)

    df = categorize_model_id(df, modelid_to_category, model_category_mapping)
    
def pre_train_run_random_forest_model(df):
    # Filter dataset to only include the last 5 years
    recent_years = df['sale_year'].dropna().unique()
    recent_years.sort()
    #selected_years = recent_years[-5:]
    #df_filtered = df[df['sale_year'].isin(selected_years)]
    df_filtered=df.copy()
    
    # Drop irrelevant columns: 'SalesID', 'saledate'
    high_cardinality_cols = [col for col in df_filtered.select_dtypes(include=['object']).columns if df_filtered[col].nunique() > 50]
    df_filtered = df_filtered.drop(columns=['SalesID', 'saledate'] + high_cardinality_cols, errors='ignore')
    categorical_cols = df_filtered.select_dtypes(include=['object']).columns


    # Identify high, moderate, and low cardinality categorical columns
    high_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() >= 50]
    moderate_cardinality_cols = [col for col in categorical_cols if 10 <= df_filtered[col].nunique() < 50]
    low_cardinality_cols = [col for col in categorical_cols if df_filtered[col].nunique() < 10]
    
    # Fill missing categorical values with "Unknown"
    for col in categorical_cols:
        df_filtered[col] = df_filtered[col].fillna("Unknown")
    
    # Encoding strategy
    
    # Apply One-Hot Encoding (OHE) to low-cardinality categories
    df_encoded = pd.get_dummies(df_filtered, columns=low_cardinality_cols, drop_first=True)
    
    # Apply Ordinal Encoding to moderate-cardinality categories
    if moderate_cardinality_cols:
        ord_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
        df_encoded[moderate_cardinality_cols] = ord_encoder.fit_transform(df_encoded[moderate_cardinality_cols])
    
    # Apply Frequency Encoding to high-cardinality categories
    for col in high_cardinality_cols:
        freq_encoding = df_encoded[col].value_counts().to_dict()
        df_encoded[col] = df_encoded[col].map(freq_encoding)
    
    # Drop rows with missing SalePrice (target variable)
    df_encoded = df_encoded.dropna(subset=['SalePrice'])
    return df_encoded

def train_model(df_encoded):
    X = df_encoded.drop(columns=['SalePrice'])
    y = df_encoded['SalePrice']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    best_params = { 'n_estimators': 100}
    
    rf_model = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)

    y_pred = rf_model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    
    # Display model performance
    print(mae, rmse)
        
    return rf_model


In [32]:
file_path_train = r"C:\Users\eitanb\Documents\DS\ML\ML_project\DATA/Train.csv"
df,product_group_avg_price,model_avg_price,modelid_to_category, model_category_mappingg=pre_train_only(file_path_train)
df_encoded=pre_train_run_random_forest_model(df)
rf_model=train_model(df_encoded)

C:\Users\eitanb\AppData\Local\Temp\ipykernel_22792\1263314632.py:96: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path_train)[['SalesID', 'SalePrice',  'ModelID',
C:\Users\eitanb\AppData\Local\Temp\ipykernel_22792\1263314632.py:76: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Predicted_Model_Category"].fillna("Unknown", inplace=True)


4540.807883070412 7310.768671236769


In [41]:
import pandas as pd

# Load validation data
Valid_df = pd.read_csv(file_path_train)[[
    'SalesID', 'ModelID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
    'saledate', 'ProductSize', 'fiProductClassDesc', 'state', 'ProductGroup',
    'Drive_System', 'Enclosure', 'Transmission', 'Turbocharged',
    'Engine_Horsepower', 'Hydraulics', 'Tire_Size', 'Track_Type',
    'Travel_Controls', 'Differential_Type', 'Steering_Controls', 'fiModelDesc'
]]

# Preserve SalesID before dropping it in preprocessing
sales_id_col = Valid_df[['SalesID']].copy()

# Apply preprocessing functions
df = update_YearMade(Valid_df)
df = saledate(df)

#model_avg_price, product_group_avg_price = pre_train(df)

# Create mappings
modelid_to_category, model_category_mapping = create_model_category_mapping(df, model_avg_price)

# Apply preprocessing
df = preprocess_data(df, modelid_to_category, model_category_mapping, product_group_avg_price)
df = Enclosure_fun(df)
df = extract_horsepower(df)
df = preprocess_product_size(df)

# Model pre-training process
df = Pre_train_test_model(df)

# Ensure index alignment
df.index = sales_id_col.index  # Keep index same as SalesID

# Make predictions
predictions = rf_model.predict(df)

# Ensure matching row count
if len(predictions) != len(df):
    raise ValueError("Mismatch in row count between processed data and predictions!")

# Construct results DataFrame with correct alignment
results = pd.DataFrame({
    "SalesID": sales_id_col["SalesID"].values,  # Retrieve original SalesID
    "SalePrice": np.exp(predictions)  # Reverse log transformation if needed
})

# Save to Excel
results.to_excel("predictions.xlsx", index=False)

C:\Users\eitanb\AppData\Local\Temp\ipykernel_22792\1187768001.py:4: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  Valid_df = pd.read_csv(file_path_train)[[


update_YearMade START


C:\Users\eitanb\AppData\Local\Temp\ipykernel_22792\908198221.py:287: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1980. 1980. 1994. ... 2003. 2004. 2004.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['YearMade'] == 1000, 'YearMade'] = df['ModelID'].map(model_medians)
C:\Users\eitanb\AppData\Local\Temp\ipykernel_22792\908198221.py:293: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['YearM

update_YearMade END


C:\Users\eitanb\AppData\Local\Temp\ipykernel_22792\908198221.py:112: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Predicted_Model_Category"].fillna("Unknown", inplace=True)
C:\Users\eitanb\AppData\Local\Temp\ipykernel_22792\908198221.py:145: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df['Engine_Horsepow

KeyboardInterrupt: 